In [1]:
%config IPCompleter.use_jedi = False
# %xmode Verbose
%xmode context
%pdb off
%load_ext viztracer
from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from typing import Dict, List, Tuple, Optional
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.mode.dtype_backend = 'pyarrow' # use new pyarrow backend instead of numpy
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()
from pyphocorehelpers.print_helpers import get_now_time_str, get_now_day_str

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field, custom_define
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin, HDF_Converter

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_subsession_neuron_differences, debug_print_ratemap, debug_print_spike_counts, debug_plot_2d_binning, print_aligned_columns
from neuropy.utils.debug_helpers import parameter_sweeps, _plot_parameter_sweep, compare_placefields_info
from neuropy.utils.indexing_helpers import union_of_arrays, intersection_of_arrays, find_desired_sort_indicies, paired_incremental_sorting

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, CapturedException, document_active_variables
from pyphocorehelpers.general_helpers import GeneratedClassDefinitionType, CodeConversion, inspect_callable_arguments

## Pho Programming Helpers:
import inspect
from pyphocorehelpers.general_helpers import inspect_callable_arguments, get_arguments_as_optional_dict, GeneratedClassDefinitionType, CodeConversion
from pyphocorehelpers.print_helpers import DocumentationFilePrinter, TypePrintMode, print_keys_if_possible, debug_dump_object_member_shapes, print_value_overview_only, document_active_variables, CapturedException
from pyphocorehelpers.programming_helpers import IPythonHelpers, PythonDictionaryDefinitionFormat, MemoryManagement
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper, print_widget_hierarchy

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation, TruncationCheckingResults # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsResult, TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer, RankOrderResult


# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')


# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations
from pyphoplacecellanalysis.SpecificResults.fourthYearPresentation import *

# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display, HTML
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, render_colors

global_data_root_parent_path = find_first_extant_path([Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Exception reporting mode: Context
Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='twolong_LR_pf1Dsession_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays. Noticed very strange jumping off the track in the 3D behavior/spikes viewer. Is there something wrong with this session?
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25')

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False
# 
# # Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=['pf_computation',
                                            #    'pfdt_computation',
                                                'firing_rate_trends',
                                                # 'pf_dt_sequential_surprise', 
                                            #    'ratemap_peaks_prominence2d',
                                                'position_decoding', 
                                                # 'position_decoding_two_step', 
                                            #    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                            #     'long_short_inst_spike_rate_groups',
                                            #     'long_short_endcap_analysis',
                                            # 'split_to_directional_laps',
]

curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'



## Post Compute Validate 2023-05-16:
was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated. This will fail because constained versions not ran yet.
if was_updated:
    print(f'was_updated: {was_updated}')
    try:
        curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
    except Exception as e:
        ## TODO: catch/log saving error and indicate that it isn't saved.
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')


basedir: /home/halechr/FastData/KDIBA/gor01/one/2006-6-08_14-26-15
Loading loaded session pickle file results : /home/halechr/FastData/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl... 

INFO:com.PhoHale.Spike3D.pipeline:NeuropyPipeline.__setstate__(state="{'pipeline_name': 'kdiba_pipeline', 'session_data_type': 'kdiba', '_stage': <pyphoplacecellanalysis.General.Pipeline.Stages.Display.DisplayPipelineStage object at 0x7fb91815e730>}")
INFO:com.PhoHale.Spike3D.pipeline:select_filters(...) with: []
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze1_odd"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze1_odd] already exists and is non-None
INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtere

done.
Loading pickled pipeline success: /home/halechr/FastData/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl.
pipeline_needs_resave but skip_save_on_initial_load == True, so saving will be skipped entirely. Be sure to save manually if there are changes.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'firing_rate_trends', 'position_decoding']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so th

INFO:com.PhoHale.Spike3D.pipeline:Performing global computations...
INFO:com.PhoHale.Spike3D.pipeline:select_filters(...) with: []
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze1_odd"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze1_odd] already exists and is non-None
INFO:com.PhoHale.Spike3D.pipeline:	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
INFO:com.PhoHale.Spike3D.pipeline:Performing perform_action_for_all_contexts with action EvaluationActions.EVALUATE_COMPUTATIONS on filtered_session with filter named "maze2_odd"...
INFO:com.PhoHale.Spike3D.pipeline:WARNING: skipping computation because overwrite_extant_results=False and active_computation_results[maze2_o

	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelis

In [3]:
### GLOBAL COMPUTATIONS:
if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results: {e}')
        raise

curr_active_pipeline.reload_default_computation_functions()

extended_computations_include_includelist=['pf_computation', 'firing_rate_trends', # 'pfdt_computation',
    # 'pf_dt_sequential_surprise',
    #  'ratemap_peaks_prominence2d',
    'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding',
    # 'long_short_rate_remapping',
    # 'long_short_inst_spike_rate_groups',
    'long_short_endcap_analysis',
    # 'spike_burst_detection',
    'split_to_directional_laps',
    'rank_order_shuffle_analysis'
] # do only specified

force_recompute_global = force_reload
# force_recompute_global = True
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if (len(newly_computed_values) > 0):
    print(f'newly_computed_values: {newly_computed_values}.')
    if (saving_mode.value != 'skip_saving'):
        print(f'Saving global results...')
        try:
            # curr_active_pipeline.global_computation_results.persist_time = datetime.now()
            # Try to write out the global computation function results:
            curr_active_pipeline.save_global_computation_results()
        except Exception as e:
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
            print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
    else:
        print(f'\n\n!!WARNING!!: changes to global results have been made but they will not be saved since saving_mode.value == "skip_saving"')
        print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
    print(f'no changes in global results.')

# except Exception as e:
#     exception_info = sys.exc_info()
#     e = CapturedException(e, exception_info)
#     print(f'second half threw: {e}')


# 4m 5.2s for inst fr computations
# Jupyter Widget Interactive
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_widget, interactive_pipeline_files

_pipeline_jupyter_widget = interactive_pipeline_widget(curr_active_pipeline=curr_active_pipeline)
# display(_pipeline_jupyter_widget)
_pipeline_jupyter_widget

Loading loaded session pickle file results : /home/halechr/FastData/KDIBA/gor01/one/2006-6-08_14-26-15/output/global_computation_results.pkl... done.
included includelist is specified: ['pf_computation', 'firing_rate_trends', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_endcap_analysis', 'split_to_directional_laps', 'rank_order_shuffle_analysis'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze_any"
pf_computation, maze_any already computed.
firing_rate_trends, maze_any already computed.
split_to_directional_laps, maze_any already computed.
rank_order_shuffle_analysis, maze_any already computed.
long_short_decoding_analyses, maze_any already computed.
short_long_pf_overlap_analyses, maze_any already computed.
long_short_fr_indicies_analyses, maze_any already computed.
jonathan_firing_rate_

In [ ]:
curr_active_pipeline.save_global_computation_results()

In [ ]:
curr_active_pipeline.export_pipeline_to_h5()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE)

# End Run

In [4]:
# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_epoch_obj, short_epoch_obj = [Epoch(curr_active_pipeline.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name.removesuffix('_any'))) for an_epoch_name in [long_epoch_name, short_epoch_name]] #TODO 2023-11-10 20:41: - [ ] Issue with getting actual Epochs from sess.epochs for directional laps: emerges because long_epoch_name: 'maze1_any' and the actual epoch label in curr_active_pipeline.sess.epochs is 'maze1' without the '_any' part.
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D

assert short_epoch_obj.n_epochs > 0, f'long_epoch_obj: {long_epoch_obj}, short_epoch_obj: {short_epoch_obj}'
assert long_epoch_obj.n_epochs > 0, f'long_epoch_obj: {long_epoch_obj}, short_epoch_obj: {short_epoch_obj}'

In [5]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global 
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global `long_short_fr_indicies_analysis`:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]
long_short_fr_indicies_df = long_short_fr_indicies_analysis_results['long_short_fr_indicies_df']

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result.Flat_epoch_time_bins_mean, expected_v_observed_result.Flat_decoder_time_bin_centers, expected_v_observed_result.num_neurons, expected_v_observed_result.num_timebins_in_epoch, expected_v_observed_result.num_total_flat_timebins, expected_v_observed_result.is_short_track_epoch, expected_v_observed_result.is_long_track_epoch, expected_v_observed_result.short_short_diff, expected_v_observed_result.long_long_diff

jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)
neuron_replay_stats_df, short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset = jonathan_firing_rate_analysis_result.get_cell_track_partitions(frs_index_inclusion_magnitude=0.05)

## Update long_exclusive/short_exclusive properties with `long_short_fr_indicies_df`
# long_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)
# short_exclusive.refine_exclusivity_by_inst_frs_index(long_short_fr_indicies_df, frs_index_inclusion_magnitude=0.5)


WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_only above
WARN: 2023-09-28 16:15: - [ ] fix the combination properties. Would work if we directly used the computed _is_L_only and _is_S_only above


In [6]:
# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']

# Most popular
# long_LR_name, short_LR_name, long_RL_name, short_RL_name, global_any_name

# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name].computation_config.pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)


In [7]:
directional_laps_results = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
included_qclu_values: float = rank_order_results.included_qclu_values
print(f'minimum_inclusion_fr_Hz: {minimum_inclusion_fr_Hz}')
print(f'included_qclu_values: {included_qclu_values}')

minimum_inclusion_fr_Hz: 1.0
included_qclu_values: [1, 2, 4, 9]


In [8]:
# NEW 2023-11-22 method: Get the templates (which can be filtered by frate first) and the from those get the decoders):        
# track_templates: TrackTemplates = directional_laps_results.get_shared_aclus_only_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # shared-only
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only
long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = track_templates.get_decoders()

# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']
# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name].computation_config.pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)

LR_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])

LR_results_long_short_z_diffs = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_long_short_z_diff = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])

In [ ]:
active_burst_intervals = curr_active_pipeline.computation_results[global_epoch_name].computed_data['burst_detection']['burst_intervals']
# active_burst_intervals

In [ ]:
# Relative Entropy/Surprise Results:
active_extended_stats = global_results['extended_stats']
active_relative_entropy_results = active_extended_stats['pf_dt_sequential_surprise'] # DynamicParameters
historical_snapshots = active_relative_entropy_results['historical_snapshots']
post_update_times: np.ndarray = active_relative_entropy_results['post_update_times'] # (4152,) = (n_post_update_times,)
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals: np.ndarray = active_relative_entropy_results['time_intervals']
surprise_time_bin_duration = (post_update_times[2]-post_update_times[1])
long_short_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['long_short_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
short_long_rel_entr_curves_frames: np.ndarray = active_relative_entropy_results['short_long_rel_entr_curves_frames'] # (4152, 108, 63) = (n_post_update_times, n_neurons, n_xbins)
flat_relative_entropy_results: np.ndarray = active_relative_entropy_results['flat_relative_entropy_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_results: np.ndarray = active_relative_entropy_results['flat_jensen_shannon_distance_results'] # (149, 63) - (nSnapshots, nXbins)
flat_jensen_shannon_distance_across_all_positions: np.ndarray = np.sum(np.abs(flat_jensen_shannon_distance_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)
flat_surprise_across_all_positions: np.ndarray = np.sum(np.abs(flat_relative_entropy_results), axis=1) # sum across all position bins # (4152,) - (nSnapshots)

## Get the placefield dt matrix:
if 'snapshot_occupancy_weighted_tuning_maps' not in active_relative_entropy_results:
	## Compute it if missing:
	occupancy_weighted_tuning_maps_over_time = np.stack([placefield_snapshot.occupancy_weighted_tuning_maps_matrix for placefield_snapshot in historical_snapshots.values()])
	active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] = occupancy_weighted_tuning_maps_over_time
else:
	occupancy_weighted_tuning_maps_over_time = active_relative_entropy_results['snapshot_occupancy_weighted_tuning_maps'] # (n_post_update_times, n_neurons, n_xbins)


In [ ]:
# Time-dependent
long_pf1D_dt, short_pf1D_dt, global_pf1D_dt = long_results.pf1D_dt, short_results.pf1D_dt, global_results.pf1D_dt
long_pf2D_dt, short_pf2D_dt, global_pf2D_dt = long_results.pf2D_dt, short_results.pf2D_dt, global_results.pf2D_dt
global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt

In [ ]:
## long_short_endcap_analysis: checks for cells localized to the endcaps that have their placefields truncated after shortening the track
truncation_checking_result: TruncationCheckingResults = curr_active_pipeline.global_computation_results.computed_data.long_short_endcap
disappearing_endcap_aclus = truncation_checking_result.disappearing_endcap_aclus
# disappearing_endcap_aclus
trivially_remapping_endcap_aclus = truncation_checking_result.minor_remapping_endcap_aclus
# trivially_remapping_endcap_aclus
significant_distant_remapping_endcap_aclus = truncation_checking_result.significant_distant_remapping_endcap_aclus
# significant_distant_remapping_endcap_aclus
appearing_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[jonathan_firing_rate_analysis_result.neuron_replay_stats_df['track_membership'] == SplitPartitionMembership.RIGHT_ONLY].index
# appearing_aclus

In [ ]:
epoch_active_aclus = np.array([9,  26,  31,  39,  40,  43,  47,  52,  53,  54,  60,  61,  65,  68,  72,  75,  77,  78,  81,  82,  84,  85,  90,  92,  93,  98, 102])


curr_active_pipeline.filtered_contexts['maze1_odd']
curr_active_pipeline.computation_results['maze1_odd']

In [ ]:
curr_active_pipeline.prepare_for_display()
curr_active_pipeline.display('_display_1d_placefields', 'maze1_odd') # , 'maze1_odd'


In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file

programmatic_render_to_file(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', write_vector_format=True, write_png=True, debug_print=True)

In [ ]:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.prepare_for_display()

In [ ]:

_out = curr_active_pipeline.display('_display_1d_placefields', 'maze_any')


In [ ]:
_out

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze_any')

In [ ]:
curr_active_pipeline.display('_display_placemaps_pyqtplot_2D', 'maze2_odd')

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze2_odd')

In [ ]:
curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', 'maze1_odd')

In [ ]:

# Adjust layout to make space for the footer
# plt.subplots_adjust(bottom=0.35)

plt.tight_layout(pad=2.0)

In [ ]:
_display_placemaps_pyqtplot_2D

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
from mpl_multitab import MplMultiTab, MplMultiTab2D
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_display_to_PDF, programmatic_render_to_file
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_single_cell_1D_placecell_validation
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1d_placecell_validations


# matplotlib_configuration_update(is_interactive=True)

# curr_active_pipeline.display('_display_grid_bin_bounds_validation')
_out = curr_active_pipeline.display('_display_1d_placefield_validations', 'maze1_odd')
_out.ui.show()

In [ ]:

programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_1d_placefield_validations', filter_name='maze1_odd', debug_print=True)

# plt.show()

In [ ]:
placefield_cell_index = 0
active_epoch_placefields1D = deepcopy(long_pf1D)
curr_cell_normalized_tuning_curve = active_epoch_placefields1D.ratemap.normalized_tuning_curves[placefield_cell_index, :].squeeze()
{'xbin_centers': active_epoch_placefields1D.ratemap.xbin_centers, 'curr_cell_normalized_tuning_curve': curr_cell_normalized_tuning_curve}

{'xbin_centers': np.array([31.0565, 34.8495, 38.6426, 42.4356, 46.2286, 50.0216, 53.8147, 57.6077, 61.4007, 65.1937, 68.9867, 72.7798, 76.5728, 80.3658, 84.1588, 87.9519, 91.7449, 95.5379, 99.3309, 103.124, 106.917, 110.71, 114.503, 118.296, 122.089, 125.882, 129.675, 133.468, 137.261, 141.054, 144.847, 148.64, 152.433, 156.226, 160.019, 163.812, 167.605, 171.398, 175.191, 178.984, 182.777, 186.57, 190.363, 194.157, 197.95, 201.743, 205.536, 209.329, 213.122, 216.915, 220.708, 224.501, 228.294, 232.087, 235.88, 239.673, 243.466, 247.259, 251.052, 254.845, 258.638, 262.431]),
 'curr_cell_normalized_tuning_curve': np.array([5.92979e-05, 0.000150933, 0.00036895, 0.000736517, 0.00121915, 0.00173714, 0.0022042, 0.00252859, 0.0026496, 0.0027108, 0.00312627, 0.00423033, 0.00579314, 0.00709557, 0.00766535, 0.00789647, 0.00884807, 0.0115452, 0.0165549, 0.0238423, 0.0323681, 0.039895, 0.0442459, 0.0452642, 0.0449909, 0.0457691, 0.0485138, 0.0525281, 0.0562324, 0.0581433, 0.0575758, 0.0544383, 0.0486438, 0.0404683, 0.0315115, 0.0243731, 0.0207242, 0.0199181, 0.0197507, 0.0183449, 0.0153819, 0.0119837, 0.00951012, 0.00827676, 0.00740415, 0.00596512, 0.00396809, 0.00210018, 0.000875453, 0.000302685, 0.000153468, 0.00027615, 0.000667689, 0.00135676, 0.00224608, 0.00305331, 0.0034339, 0.0031979, 0.0024518, 0.00153458, 0.00079294, 0.000405152])}


In [ ]:
# batch_extended_programmatic_figures


In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze1_odd')

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze2_even')

In [ ]:

#TODO 2023-11-29 09:18: - [ ] Not good, the self.filtered_contexts are not unique!
list(curr_active_pipeline.filtered_contexts.values())
# [IdentifyingContext<(... 'maze2')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(..., 'maze')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(..., 'maze')>, IdentifyingContext<(...ze1_any')>, IdentifyingContext<(... 'maze2')>, IdentifyingContext<(..., 'maze')>]
[(v == curr_active_pipeline.filtered_contexts['maze1_even']) for v in list(curr_active_pipeline.filtered_contexts.values())]
# [True, True, False, True, True, False, False, True, False]
# meaning `curr_active_pipeline.display('_display_1d_placefields', curr_active_pipeline.filtered_contexts['maze1_even'])` doesn't work
curr_active_pipeline.filtered_contexts.index(curr_active_pipeline.filtered_contexts['maze1_even'])

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze2_odd')


In [ ]:
write_vector_format = False
write_png = True
debug_print = True
from neuropy.plotting.ratemaps import BackgroundRenderingOptions

programmatic_render_to_file(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D', write_vector_format=write_vector_format, write_png=write_png, debug_print=debug_print, bg_rendering_mode=BackgroundRenderingOptions.EMPTY) #  🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.


In [ ]:
_out = curr_active_pipeline.display('_display_2d_placefield_occupancy', 'maze2_any')

In [ ]:
_out = curr_active_pipeline.display('_display_2d_placefield_occupancy', 'maze1_any')
occupancy_ax = _out.axes #.get_aspect()
pf = long_pf2D
# pf.xbin
# pf.ybin
# pf.xbin_centers
# pf.ybin_centers

# aspect_ratio = np.ptp(pf.xbin) / np.ptp(pf.ybin)  # ptp: peak to peak (range)
# aspect_ratio = 0.102803738317757
# print(f'aspect_ratio: {aspect_ratio}')
# occupancy_ax.set_aspect(aspect_ratio, adjustable='box') # If 'box', change the physical dimensions of the Axes. If 'datalim', change the x or y data limits.


## See 
# https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_anchor.html#


occupancy_ax.set_aspect('equal', adjustable=None)


In [ ]:
occupancy_ax.set_aspect('equal', adjustable='datalim')

In [ ]:
occupancy_ax.set_aspect('equal', adjustable='box')

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
programmatic_render_to_file(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_occupancy', write_vector_format=write_vector_format, write_png=write_png, debug_print=debug_print)

# EVEN: "RL", ODD: "LR"
Starts with Even (idx=0)
- EVEN: "RL"
shared_RL_aclus_only_neuron_IDs
`is_even = (an_epoch.lap_dir == 0)`
- ODD: "LR"
shared_LR_aclus_only_neuron_IDs
`is_odd = (an_epoch.lap_dir == 1)`

# 🟢 2023-10-20 - Z-Score Comparisons with Neuron_ID Shuffled templates
1. Take the intersection of the long and short templates to get only the common cells
2. Determine the long and short "tempaltes": this is done by ranking the aclus for each by their placefields' center of mass. `compute_placefield_center_of_masses`
	2a. `long_pf_peak_ranks`, `short_pf_peak_ranks` - there are one of each of these for each shared aclu.
3. Generate the unit_id shuffled (`shuffled_aclus`, `shuffle_IDXs`) ahead of time to use to shuffle the two templates during the epochs.
4. For each replay event, take each shuffled template
	4a. Iterate through each shuffle and obtain the shuffled templates like `long_pf_peak_ranks[epoch_specific_shuffled_indicies]`, `short_pf_peak_ranks[epoch_specific_shuffled_indicies]`
	4b. compute the spearman rank-order of the event and each shuffled template, and accumulate the results in `long_spearmanr_rank_stats_results`, `short_spearmanr_rank_stats_results`

5. After we're done with the shuffle loop, accumulate the results and convert to the right output format.

6. When all epochs are done, loop through the results (the epochs again) and compute the z-scores for each epoch so they can be compared to each other. Keep track of the means and std_dev for comparisons later, and subtract the two sets of z-scores (long/short) to get the delta_Z for each template.

7. TODO: Next figure out what to do with the array of z-scores and delta_Z. We have:
	n_epochs sets of results
		n_shuffles scores of delta_Z



## Convo with Kamran 2023-10-23:
- Use directional templates **
- No need to worry about re-ranking
[X] Plot the long and short separately in addition to the difference, so we show significant reqplay on each as a sanity check
[X] Absolute value difference?
[X] Fisher transform the correlation values (check if there is a difference) because correlation coefficients aren't going to be normally distributed.
	[ ] Then Z-score releative to fisher.

- T-test to compare to mean of zero (if looking at the difference)

In [ ]:
## Concerns:
# 1. Permutation recommended over shuffling for small numbers of ids
# 2.

# 5Hz thresholding of templates


In [9]:
from nptyping import NDArray
from attrs import define, field, Factory, astuple
import scipy.stats
from scipy import ndimage
from neuropy.utils.misc import build_shuffled_ids # used in _SHELL_analyze_leave_one_out_decoding_results
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_paired_t_test


# minimum_inclusion_fr_Hz: float = 2.0
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz

# Recover from the saved global result:
directional_laps_results = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
# NEW 2023-11-22 method: Get the templates (which can be filtered by frate first) and the from those get the decoders):        
# track_templates: TrackTemplates = directional_laps_results.get_shared_aclus_only_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # shared-only
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only
long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = track_templates.get_decoders()

## Pre 2023-11-22 method: building a TrackTemplates object after getting the raw decoders:
# long_LR_one_step_decoder_1D, long_RL_one_step_decoder_1D, short_LR_one_step_decoder_1D, short_RL_one_step_decoder_1D = directional_laps_results.get_decoders()
# long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = directional_laps_results.get_shared_aclus_only_decoders()
# track_templates: TrackTemplates = TrackTemplates.init_from_paired_decoders(LR_decoder_pair=(long_LR_decoder, short_LR_decoder), RL_decoder_pair=(long_RL_decoder, short_RL_decoder))
# # track_templates: TrackTemplates = TrackTemplates.init_from_paired_decoders(LR_decoder_pair=(long_LR_one_step_decoder_1D, short_LR_one_step_decoder_1D), RL_decoder_pair=(long_RL_one_step_decoder_1D, short_RL_one_step_decoder_1D)) # NOTE: now use the un-constrained versions

# Unpack all directional variables:
## {"even": "RL", "odd": "LR"}
long_LR_name, short_LR_name, global_LR_name, long_RL_name, short_RL_name, global_RL_name, long_any_name, short_any_name, global_any_name = ['maze1_odd', 'maze2_odd', 'maze_odd', 'maze1_even', 'maze2_even', 'maze_even', 'maze1_any', 'maze2_any', 'maze_any']
# Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
(long_LR_context, long_RL_context, short_LR_context, short_RL_context) = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
long_LR_epochs_obj, long_RL_epochs_obj, short_LR_epochs_obj, short_RL_epochs_obj, global_any_laps_epochs_obj = [curr_active_pipeline.computation_results[an_epoch_name].computation_config.pf_params.computation_epochs for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name, global_any_name)] # note has global also
(long_LR_session, long_RL_session, short_LR_session, short_RL_session) = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)] # sessions are correct at least, seems like just the computation parameters are messed up
(long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name].computed_data for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_computation_config, long_RL_computation_config, short_LR_computation_config, short_RL_computation_config) = [curr_active_pipeline.computation_results[an_epoch_name].computation_config for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
(long_LR_pf1D, long_RL_pf1D, short_LR_pf1D, short_RL_pf1D) = (long_LR_results.pf1D, long_RL_results.pf1D, short_LR_results.pf1D, short_RL_results.pf1D)
(long_LR_pf2D, long_RL_pf2D, short_LR_pf2D, short_RL_pf2D) = (long_LR_results.pf2D, long_RL_results.pf2D, short_LR_results.pf2D, short_RL_results.pf2D)
(long_LR_pf1D_Decoder, long_RL_pf1D_Decoder, short_LR_pf1D_Decoder, short_RL_pf1D_Decoder) = (long_LR_results.pf1D_Decoder, long_RL_results.pf1D_Decoder, short_LR_results.pf1D_Decoder, short_RL_results.pf1D_Decoder)

LR_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_real_values = np.array([(long_stats_z_scorer.real_value, short_stats_z_scorer.real_value) for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])

LR_results_long_short_z_diffs = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.LR_ripple.ranked_aclus_stats_dict.items()])
RL_results_long_short_z_diff = np.array([long_short_z_diff for epoch_id, (long_stats_z_scorer, short_stats_z_scorer, long_short_z_diff, long_short_naive_z_diff, is_forward_replay) in rank_order_results.RL_ripple.ranked_aclus_stats_dict.items()])


# LR_results_real_values

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import plot_rank_order_histograms

# Plot histograms:
post_title_info: str = f'{minimum_inclusion_fr_Hz} Hz\n{curr_active_pipeline.get_session_context().get_description()}'
_out_z_score, _out_real, _out_most_likely_z = plot_rank_order_histograms(rank_order_results, post_title_info=post_title_info)


#TODO 2023-12-10 19:56: - [ ] Histogram Display Helpers

#TODO 2023-12-10 19:56: - [ ] Pf1D Helpers

#TODO 2023-12-10 19:57: - [ ] Variant Saving
 

In [ ]:
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=0.0) # non-shared-only
long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder = track_templates.get_decoders()

# filtered_decoder_list = [filtered_by_frate(a_decoder, minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, debug_print=True) for a_decoder in (long_LR_one_step_decoder_1D, long_RL_one_step_decoder_1D, short_LR_one_step_decoder_1D, short_RL_one_step_decoder_1D)]
original_neuron_ids_list = [a_decoder.pf.ratemap.neuron_ids for a_decoder in (long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder)]
is_aclu_included_list = [a_decoder.pf.ratemap.tuning_curve_unsmoothed_peak_firing_rates >= minimum_inclusion_fr_Hz for a_decoder in (long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder)]
filtered_aclus_list = [np.array(a_decoder.pf.ratemap.neuron_ids)[a_decoder.pf.ratemap.tuning_curve_unsmoothed_peak_firing_rates >= minimum_inclusion_fr_Hz] for a_decoder in (long_LR_decoder, long_RL_decoder, short_LR_decoder, short_RL_decoder)]

## For a given run direction (LR/RL) let's require inclusion in either (OR) long v. short to be included.
filtered_included_LR_aclus = np.union1d(filtered_aclus_list[0], filtered_aclus_list[2])
filtered_included_RL_aclus = np.union1d(filtered_aclus_list[1], filtered_aclus_list[3])
# build the final shared aclus:
filtered_direction_shared_aclus_list = [filtered_included_LR_aclus, filtered_included_RL_aclus, filtered_included_LR_aclus, filtered_included_RL_aclus] # contains the shared aclus for that direction
# rebuild the is_aclu_included_list from the shared aclus
is_aclu_included_list = [np.isin(an_original_neuron_ids, a_filtered_neuron_ids) for an_original_neuron_ids, a_filtered_neuron_ids in zip(original_neuron_ids_list, filtered_direction_shared_aclus_list)]

# is_aclu_included_list[0]
filtered_direction_shared_aclus_list

In [ ]:
# # for 5Hz:
# [array([  5,   7,  31,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  95,  99, 100, 108]),
#  array([  5,   7,   9,  31,  32,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  93,  95,  99, 101, 108]),
#  array([  5,   7,  31,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  95,  99, 100, 108]),
#  array([  5,   7,   9,  31,  32,  39,  41,  45,  46,  48,  50,  55,  61,  62,  64,  69,  72,  75,  76,  78,  79,  83,  84,  86,  88,  90,  91,  92,  93,  95,  99, 101, 108])]

# # for 20Hz:
# [array([  5,  41,  46,  48,  69,  78,  79,  83,  86,  88,  90, 108]),
#  array([ 62,  64,  75,  78,  83,  91, 101]),
#  array([  5,  41,  46,  48,  69,  78,  79,  83,  86,  88,  90, 108]),
#  array([ 62,  64,  75,  78,  83,  91, 101])]

# 2023-11-22 - RECOMPUTE

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalLapsHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderGlobalComputationFunctions

curr_active_pipeline.reload_default_computation_functions()

## clear the old values to prepare for the new ones:
curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'] = None
curr_active_pipeline.global_computation_results.computed_data['RankOrder'] = None
del curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
del curr_active_pipeline.global_computation_results.computed_data['RankOrder']


In [ ]:
track_templates.any_decoder_neuron_IDs
track_templates.decoder_neuron_IDs_list

In [ ]:
active_spikes_df = deepcopy(spikes_df).spikes.sliced_by_neuron_id(track_templates.any_decoder_neuron_IDs)
active_spikes_df.spikes.adding_epochs_identity_column()

In [ ]:

curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'] = DirectionalLapsHelpers.build_global_directional_result_from_natural_epochs(curr_active_pipeline, progress_print=True) # repalce the directional laps object
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']

# minimum_inclusion_fr_Hz = 1.0
# included_qclu_values = [1,2]

minimum_inclusion_fr_Hz = 1.0
included_qclu_values = [1,2,4,9]

# perform_rank_order_shuffle_analysis
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-perform_rank_order_shuffle_analysis_{curr_active_pipeline.session_name}.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
RankOrderGlobalComputationFunctions.perform_rank_order_shuffle_analysis(curr_active_pipeline, curr_active_pipeline.global_computation_results, None, None, include_includelist=None, debug_print=True,
                                                                        num_shuffles=100, minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, included_qclu_values=included_qclu_values, skip_laps=False)


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses

directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
selected_spikes_df = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.selected_spikes_df)
# active_epochs = global_computation_results.computed_data['RankOrder'].ripple_most_likely_result_tuple.active_epochs
active_epochs = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.epochs_df)
track_templates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz)
ripple_combined_epoch_stats_df, ripple_new_output_tuple = RankOrderAnalyses.pandas_df_based_correlation_computations(selected_spikes_df=selected_spikes_df, active_epochs_df=active_epochs, track_templates=track_templates, num_shuffles=5)

In [ ]:
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
included_qclu_values: List[int] = rank_order_results.included_qclu_values
ripple_result_tuple, laps_result_tuple = rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only -- !! Is minimum_inclusion_fr_Hz=None the issue/difference?
print(f'minimum_inclusion_fr_Hz: {minimum_inclusion_fr_Hz}')
print(f'included_qclu_values: {included_qclu_values}')

# 10m 29.5s for 1000 shuffles.  c:\Users\pho\repos\Spike3DWorkEnv\Spike3D\viztracer_2023-11-22_16-11-perform_rank_order_shuffle_analysis.json

# 3m 33.9s - 500
# 3m 26.4s - 1000

In [ ]:
_ripples_outputs = RankOrderAnalyses.main_ripples_analysis(curr_active_pipeline, num_shuffles=500, rank_alignment='median', minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, included_qclu_values=included_qclu_values)
(LR_ripple_outputs, RL_ripple_outputs, ripple_evts_paired_tests) = _ripples_outputs


In [ ]:
# LR_ripple_outputs.epochs_df
LR_ripple_outputs.spikes_df

In [ ]:
RL_ripple_outputs.spikes_df

In [ ]:
LR_ripple_outputs.selected_spikes_df

In [ ]:
RL_ripple_outputs.



In [ ]:
## Ensure equivalence of the two LR_ripple_outputs and RL_ripple_outputs for the fields that matter:
assert LR_ripple_outputs.spikes_df.equals(RL_ripple_outputs.spikes_df), f"spikes_df are not equal"
assert LR_ripple_outputs.selected_spikes_df.equals(RL_ripple_outputs.selected_spikes_df), f"selected_spikes_df are not equal"
assert LR_ripple_outputs.epochs_df.equals(RL_ripple_outputs.epochs_df), f"epochs_df are not equal"

In [ ]:
_new_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(deepcopy(LR_ripple_outputs.selected_spikes_df), deepcopy(LR_ripple_outputs.epochs_df), track_templates, rank_order_results, None, None)



In [ ]:
# TypeError: <lambda>() missing 1 required positional argument


In [ ]:
## Recompute just the `most_likely_directional_rank_order_shuffling` part:
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import DirectionalRankOrderLikelihoods
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses
## Main
ripple_result_tuple, laps_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.003)
rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple = ripple_result_tuple, laps_result_tuple

In [ ]:
directional_likelihoods_tuple: DirectionalRankOrderLikelihoods = deepcopy(ripple_result_tuple.directional_likelihoods_tuple)
directional_likelihoods_tuple.long_best_direction_indices
directional_likelihoods_tuple.short_best_direction_indices
# directional_likelihoods_tuple.long_relative_direction_likelihoods





# Saving/Loading `DirectionalLaps_2Hz`

In [ ]:
# 2023-11-27 - I'd like to be able to save/load single results a time, (meaning specific to their parameters):
# day_date_str: str = '2023-12-11-minimum_inclusion_fr_Hz_2_included_qclu_values_1-2_'
class SaveStringGenerator:
	_minimal_decimals_float_formatter = lambda x: f"{x:.1f}".rstrip('0').rstrip('.')
	
def generate_save_suffix(minimum_inclusion_fr_Hz: float, included_qclu_values: List[int], day_date: str='2023-12-11') -> str:
	# day_date_str: str = '2023-12-11-minimum_inclusion_fr_Hz_2_included_qclu_values_1-2_'
	print(f'minimum_inclusion_fr_Hz: {minimum_inclusion_fr_Hz}')
	print(f'included_qclu_values: {included_qclu_values}')
	out_filename_str: str = '-'.join([day_date, f'minimum_inclusion_fr', SaveStringGenerator._minimal_decimals_float_formatter(minimum_inclusion_fr_Hz), f'included_qclu_values', f'{included_qclu_values}'])
	return out_filename_str

# list = ['2Hz', '12Hz']

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData
from attrs import asdict
## Uses `generate_save_suffix` and the current rank_order_result's parameters to build a reasonable save name:
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
included_qclu_values: List[int] = rank_order_results.included_qclu_values
out_filename_str = generate_save_suffix(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, included_qclu_values=included_qclu_values, day_date='2023-12-18_720pm')
print(f'out_filename_str: "{out_filename_str}"')
directional_laps_output_path = curr_active_pipeline.get_output_path().joinpath(f'{out_filename_str}DirectionalLaps.pkl').resolve()
saveData(directional_laps_output_path, (curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']))
rank_order_output_path = curr_active_pipeline.get_output_path().joinpath(f'{out_filename_str}RankOrder.pkl').resolve()
saveData(rank_order_output_path, (curr_active_pipeline.global_computation_results.computed_data['RankOrder']))
# saveData(rank_order_output_path, (asdict(curr_active_pipeline.global_computation_results.computed_data['RankOrder'], recurse=True)))

# saveData(directional_laps_output_path, (curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'], asdict(curr_active_pipeline.global_computation_results.computed_data['RankOrder'], recurse=False))) 
# saveData(directional_laps_output_path, (curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']))

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData

# Load the data from a file into the pipeline:
# out_filename_str: str = '2023-12-11-minimum_inclusion_fr_Hz_2_included_qclu_values_1-2_' # specific

minimum_inclusion_fr_Hz: float = 2.0
included_qclu_values: List[int] = [1,2,4,9]
out_filename_str = generate_save_suffix(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz, included_qclu_values=included_qclu_values, day_date='2023-12-13_1000')
print(f'out_filename_str: "{out_filename_str}"')
# day_date_str: str = '2023-12-11_with_tuple_newer_'
# day_date_str: str = ''
directional_laps_output_path = curr_active_pipeline.get_output_path().joinpath(f'{out_filename_str}DirectionalLaps.pkl').resolve()
assert directional_laps_output_path.exists()
# loaded_directional_laps, loaded_rank_order = loadData(directional_laps_output_path)
loaded_directional_laps = loadData(directional_laps_output_path)
assert (loaded_directional_laps is not None)
# assert (loaded_rank_order is not None)
# Apply the loaded data to the pipeline:
# curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'], curr_active_pipeline.global_computation_results.computed_data['RankOrder'] = loaded_directional_laps, loaded_rank_order
# curr_active_pipeline.global_computation_results.computed_data['RankOrder']


In [ ]:
rank_order_output_path = curr_active_pipeline.get_output_path().joinpath(f'{out_filename_str}RankOrder.pkl').resolve()
loaded_rank_order = loadData(rank_order_output_path)
loaded_rank_order
# asdict(loaded_directional_laps, recurse=True) == asdict(curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps'], recurse=True)

In [ ]:
rank_order_results.RL_ripple.selected_spikes_df

In [ ]:
rank_order_results.LR_ripple.selected_spikes_df

# POST-Compute:

In [10]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalDisplayFunctions
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multi_sort_raster_browser
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import RankOrderRastersDebugger

from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import paired_separately_sort_neurons, paired_incremental_sort_neurons # _display_directional_template_debugger
from neuropy.utils.indexing_helpers import paired_incremental_sorting, union_of_arrays, intersection_of_arrays, find_desired_sort_indicies
from pyphoplacecellanalysis.GUI.Qt.Widgets.ScrollBarWithSpinBox.ScrollBarWithSpinBox import ScrollBarWithSpinBox

from neuropy.utils.mixins.HDF5_representable import HDF_SerializationMixin
from pyphoplacecellanalysis.General.Model.ComputationResults import ComputedResult
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import TrackTemplates
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses, RankOrderResult, ShuffleHelper, Zscorer, LongShortStatsTuple, DirectionalRankOrderLikelihoods, DirectionalRankOrderResult, RankOrderComputationsContainer
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import TimeColumnAliasesProtocol
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer

## Display Testing
# from pyphoplacecellanalysis.External.pyqtgraph import QtGui
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import pyqtplot_build_image_bounds_extent, pyqtplot_plot_image


spikes_df = curr_active_pipeline.sess.spikes_df
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']
minimum_inclusion_fr_Hz: float = rank_order_results.minimum_inclusion_fr_Hz
included_qclu_values: List[int] = rank_order_results.included_qclu_values
ripple_result_tuple, laps_result_tuple = rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz) # non-shared-only -- !! Is minimum_inclusion_fr_Hz=None the issue/difference?
print(f'minimum_inclusion_fr_Hz: {minimum_inclusion_fr_Hz}')
print(f'included_qclu_values: {included_qclu_values}')
# ripple_result_tuple

## Unpacks `rank_order_results`: 
# global_replays = Epoch(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
# global_replays = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
# active_replay_epochs, active_epochs_df, active_selected_spikes_df = combine_rank_order_results(rank_order_results, global_replays, track_templates=track_templates)
# active_epochs_df

# 2023-12-15 - Newest method:
laps_combined_epoch_stats_df = rank_order_results.laps_combined_epoch_stats_df
ripple_combined_epoch_stats_df = rank_order_results.ripple_combined_epoch_stats_df
ripple_combined_epoch_stats_df


minimum_inclusion_fr_Hz: 1.0
included_qclu_values: [1, 2, 4, 9]


,LR_Long_spearman,RL_Long_spearman,LR_Short_spearman,RL_Short_spearman,LR_Long_pearson,RL_Long_pearson,LR_Short_pearson,RL_Short_pearson,LR_Long_spearman_Z,RL_Long_spearman_Z,LR_Short_spearman_Z,RL_Short_spearman_Z,LR_Long_pearson_Z,RL_Long_pearson_Z,LR_Short_pearson_Z,RL_Short_pearson_Z,label
0,-0.190484,-0.505266,-0.311799,-0.084962,-0.309079,-0.597488,-0.300516,-0.190023,-0.590383,-1.553697,-0.958812,-0.257787,-0.954941,-1.837996,-0.923088,-0.584981,0.0
1,-0.561051,-0.623965,-0.351405,-0.553211,-0.519558,-0.374148,-0.160426,-0.362877,-1.742419,-1.920624,-1.081455,-1.706578,-1.608038,-1.147890,-0.488981,-1.120332,1.0
2,-0.553194,-0.825723,-0.808135,-0.596849,-0.476285,-0.480148,-0.750906,-0.615324,-1.717995,-2.544304,-2.495750,-1.841595,-1.473765,-1.475422,-2.318747,-1.902192,2.0
3,0.110535,-0.022100,-0.137383,-0.288671,-0.220142,-0.422706,-0.245525,-0.663584,0.345439,-0.060114,-0.418720,-0.888072,-0.678978,-1.297932,-0.752682,-2.051661,NaN
4,-0.636112,-0.780502,-0.798863,-0.750366,-0.575959,-0.603304,-0.745096,-0.694433,-1.975775,-2.404517,-2.467038,-2.316587,-1.783043,-1.855967,-2.300744,-2.147204,4.0
5,-0.483333,-0.216667,0.309626,-0.158997,-0.527548,-0.418233,0.537471,-0.404880,-1.500808,-0.661566,0.965473,-0.486855,-1.632828,-1.284108,1.673649,-1.250422,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,-0.254545,-0.182650,-0.278483,0.123289,-0.142820,-0.036904,-0.146796,0.107877,-0.789541,-0.556414,-0.855647,0.386555,-0.439056,-0.105824,-0.446742,0.337652,545.0
546,-0.726035,-0.375295,-0.753436,-0.270485,-0.607512,-0.327355,-0.585654,-0.318592,-2.255331,-1.151924,-2.326371,-0.831806,-1.880949,-1.003303,-1.806668,-0.983176,546.0
547,0.164134,-0.233874,-0.229885,-0.244660,0.185153,-0.147929,-0.296207,-0.357092,0.512071,-0.714758,-0.705159,-0.751900,0.578612,-0.448885,-0.909737,-1.102416,547.0


In [ ]:
np.logical_not(np.isnan(rank_order_results.ripple_combined_epoch_stats_df.index).any())


ripple_combined_epoch_stats_df.label.isna()

In [ ]:
ripple_combined_epoch_stats_df

In [ ]:
np.isnan(ripple_combined_epoch_stats_df.label).any()

In [ ]:
np.isnan(ripple_combined_epoch_stats_df.index).any()

In [ ]:
print(f'\tdone. building global result.')
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
selected_spikes_df = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.selected_spikes_df)
# active_epochs = global_computation_results.computed_data['RankOrder'].ripple_most_likely_result_tuple.active_epochs
active_epochs = deepcopy(curr_active_pipeline.global_computation_results.computed_data['RankOrder'].LR_ripple.epochs_df)
track_templates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=minimum_inclusion_fr_Hz)
ripple_combined_epoch_stats_df, ripple_new_output_tuple = RankOrderAnalyses.pandas_df_based_correlation_computations(selected_spikes_df=selected_spikes_df, active_epochs_df=active_epochs, track_templates=track_templates, num_shuffles=150)
# new_output_tuple (output_active_epoch_computed_values, valid_stacked_arrays, real_stacked_arrays, n_valid_shuffles) = ripple_new_output_tuple
curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_combined_epoch_stats_df, curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_new_output_tuple = ripple_combined_epoch_stats_df, ripple_new_output_tuple
print(f'done!')

In [ ]:
laps_combined_epoch_stats_df

In [ ]:
print_object_memory_usage(output_active_epoch_computed_values) # 0.946189 MB


In [ ]:
## #TODO 2023-12-13 02:07: - [ ] Figure out how 'Probe_Epoch_id' maps to `ripple_result_tuple.active_epochs`
ripple_result_tuple.active_epochs
rank_order_results.LR_ripple.ranked_aclus_stats_dict


In [ ]:
## Add the pf_x information for each aclu:
## 2023-10-11 - Get the long/short peak locations
# decoder_peak_coms_list = [a_decoder.pf.ratemap.peak_tuning_curve_center_of_masses[is_good_aclus] for a_decoder in decoder_args]
decoder_aclu_peak_location_dict_list = [dict(zip(neuron_IDs, peak_locations)) for neuron_IDs, peak_locations in zip(track_templates.decoder_neuron_IDs_list, track_templates.decoder_peak_location_list)]
decoder_aclu_peak_location_dict_list


In [ ]:
track_templates.long_LR_decoder.peak_locations

In [ ]:
track_templates.decoder_LR_pf_peak_ranks_list

In [ ]:
## Replays:
global_replays = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
if isinstance(global_replays, pd.DataFrame):
	global_replays = Epoch(global_replays.epochs.get_valid_df())

# get the aligned epochs and the z-scores aligned to them:
active_replay_epochs, (active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score) = rank_order_results.get_aligned_events(global_replays.to_dataframe().copy(), is_laps=False)
active_replay_epochs

In [ ]:
## Laps:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
global_laps = deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].laps).trimmed_to_non_overlapping()
active_laps_epochs, (active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score) = rank_order_results.get_aligned_events(global_laps.to_dataframe(), is_laps=True)

In [ ]:
ripple_result_tuple.plot_histogram()

In [ ]:
# Find only the significant events (|z| > 1.96):
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses

filtered_z_score_df, (n_events, n_significant_events, percent_significant_events) = RankOrderAnalyses.find_only_significant_events(rank_order_results, high_z_criteria=1.96)
filtered_z_score_df

In [ ]:
print(filtered_z_score_df.index.to_numpy())


In [ ]:
# 2023-11-20 - Finding high-significance periods for Kamran:
z_threshold = 1.96
is_greater_than_z_threshold_long = (np.abs(ripple_result_tuple.long_best_dir_z_score_values) > z_threshold)
is_greater_than_z_threshold_short = (np.abs(ripple_result_tuple.short_best_dir_z_score_values) > z_threshold)
is_significant_either = np.logical_or(is_greater_than_z_threshold_long, is_greater_than_z_threshold_short)
is_significant_either

# is_greater_than_3std_long = (np.abs(ripple_result_tuple.long_best_dir_z_score_values) >= 3.0)
# is_greater_than_3std_short = (np.abs(ripple_result_tuple.short_best_dir_z_score_values) >= 3.0)
# is_significant_either = np.logical_or(is_greater_than_3std_long, is_greater_than_3std_short)


In [ ]:
significant_ripple_epochs = deepcopy(Epoch(ripple_result_tuple.active_epochs)).boolean_indicies_slice(is_significant_either)
# significant_ripple_epochs = deepcopy(global_replays).boolean_indicies_slice(is_significant_either)
significant_ripple_epochs.to_dataframe()

# significant_ripple_epochs.filename = Path(f'output/2023-11-27_SignificantReplayRipples').resolve()
# significant_ripple_epochs.to_neuroscope()


In [ ]:
# active_epochs = ripple_result_tuple.active_epochs
active_epochs: Epoch = rank_order_results.RL_ripple.epochs_df # Epoch(rank_order_results.RL_ripple.epochs_df)
# type(active_epochs)
active_epochs.n_epochs
# rank_order_results.RL_ripple.spikes_df

In [ ]:
rank_order_results.LR_ripple.epochs_df
rank_order_results.LR_ripple.spikes_df



In [ ]:
combined_variable_names: ['LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson']
combined_variable_z_score_column_names: ['LR_Long_spearman_Z', 'RL_Long_spearman_Z', 'LR_Short_spearman_Z', 'RL_Short_spearman_Z', 'LR_Long_pearson_Z', 'RL_Long_pearson_Z', 'LR_Short_pearson_Z', 'RL_Short_pearson_Z']

        


In [ ]:
curr_active_pipeline.build_display_context_for_filtered_session(filtered_session_name='maze_any', display_fn_name='test')

In [ ]:
rank_order_results.LR_ripple.selected_spikes_df

In [ ]:
rank_order_results.RL_ripple.selected_spikes_df

# `RankOrderRastersDebugger`

In [23]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import RankOrderRastersDebugger
from pyphocorehelpers.gui.Qt.color_helpers import build_adjusted_color
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import a_debug_callback_fn, debug_update_long_short_info_titles, debug_update_plot_titles

curr_active_pipeline.reload_default_display_functions()

## RankOrderRastersDebugger: 
# _out_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, ripple_result_tuple.active_epochs, track_templates, rank_order_results.RL_ripple.selected_spikes_fragile_linear_neuron_IDX_dict, rank_order_results.LR_ripple.selected_spikes_fragile_linear_neuron_IDX_dict)

## Required prereqs:
active_epochs_df = deepcopy(rank_order_results.LR_ripple.epochs_df)
# active_selected_spikes_df, active_epochs_df = RankOrderAnalyses.new_compute_correlations(selected_spikes_df=selected_spikes_df, active_epochs=active_epochs, track_templates=track_templates)
## 2023-12-13 11am - uses `active_epochs_df` passed in that has been augmented with all the old results:
# _out_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, active_epochs_df, track_templates, rank_order_results, rank_order_results.RL_ripple.selected_spikes_fragile_linear_neuron_IDX_dict, rank_order_results.LR_ripple.selected_spikes_fragile_linear_neuron_IDX_dict)
_out_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, active_epochs_df, track_templates, rank_order_results, rank_order_results.RL_ripple.selected_spikes_df, rank_order_results.LR_ripple.selected_spikes_df)
_out_rank_order_event_raster_debugger.params.enable_show_Z_values = False


[autoreload of pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations failed: Traceback (most recent call last):
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 274, in check
    superreload(m, reload, self.old_objects, self.shell)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setattr__(r

included_neuron_ids: [  5   7   9  31  32  36  38  39  41  45  46  48  50  51  53  55  57  59  61  62  63  64  69  72  75  76  78  79  83  84  86  88  89  90  91  92  93  95  98  99 100 101 102 108], n_neurons: 44
unit_sort_order: [39 33 35 28  1  8 29 34 12 26 37 30  0 27  5 25 11  6 21 32 14 24  3 20 22 17 23  7 13 38 18 16 40  4  9 10 15 31 36  2 19]
desired_sort_arr: [102  91  95  84   7  45  86  92  51  79  99  88   5  83  38  78  50  39  69  90  55  76  31  64  72  61  75  41  53 100  62  59 108  36  46  48  57  89  98   9  63]
WARN: len(neuron_colors): 44 > n_cells: 41: restricting neuron_colors to the correct aclus, but if colors ever get off this is where it is happening!
merged_kwargs: {'name': 'spikeRasterOverviewWindowScatterPlotItem', 'pxMode': True, 'symbol': <PyQt5.QtGui.QPainterPath object at 0x7fb480e29b30>, 'size': 2, 'pen': {'color': 'w', 'width': 1}, 'hoverable': True}
unit_sort_order: [36  1  9 15 25 29  8 32  6 30 31  3 34  4 19 35  7 11 12 24 28 17 21 22 18 20 23

emitChanged(): self.val: 0
valueChanged(new_val: 0)
an_epoch: EpochTuple(Index=0, start=40.18717686412856, stop=40.48755687032826, label=0, duration=0.3003800061997026, LR_Long_ActuallyIncludedAclus=array([ 32,  36,  41,  45,  48,  53,  59,  62,  63,  78, 102, 108]), LR_Long_rel_num_cells=12, RL_Long_ActuallyIncludedAclus=array([ 32,  36,  41,  45,  48,  53,  59,  62,  63,  78, 102, 108]), RL_Long_rel_num_cells=12, LR_Short_ActuallyIncludedAclus=array([ 32,  36,  41,  45,  48,  53,  59,  62,  63,  78, 102, 108]), LR_Short_rel_num_cells=12, RL_Short_ActuallyIncludedAclus=array([ 32,  36,  41,  45,  48,  53,  59,  62,  63,  78, 102, 108]), RL_Short_rel_num_cells=12, LR_evidence=6.9673612122637785, RL_evidence=5.032638787736222, LR_product_evidence=0.0, RL_product_evidence=0.0, normed_LR_evidence=0.5806134343553149, normed_RL_evidence=0.4193865656446852, normed_product_LR_evidence=nan, normed_product_RL_evidence=nan)
an_epoch: EpochTuple(Index=0, start=40.18717686412856, stop=40.487556870

In [ ]:
pxMode=False


In [24]:
_out_rank_order_selected_spikes_only_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(deepcopy(rank_order_results.LR_ripple.selected_spikes_df), deepcopy(rank_order_results.LR_ripple.epochs_df), track_templates, rank_order_results, None, None) # rank_order_results.RL_ripple.selected_spikes_fragile_linear_neuron_IDX_dict, rank_order_results.LR_ripple.selected_spikes_fragile_linear_neuron_IDX_dict)


included_neuron_ids: [  5   7   9  31  32  36  38  39  41  45  46  48  50  51  53  55  57  59  61  62  63  64  69  72  75  76  78  79  83  84  86  88  89  90  91  92  93  95  98  99 100 101 102 108], n_neurons: 44
unit_sort_order: [39 33 35 28  1  8 29 34 12 26 37 30  0 27  5 25 11  6 21 32 14 24  3 20 22 17 23  7 13 38 18 16 40  4  9 10 15 31 36  2 19]
desired_sort_arr: [102  91  95  84   7  45  86  92  51  79  99  88   5  83  38  78  50  39  69  90  55  76  31  64  72  61  75  41  53 100  62  59 108  36  46  48  57  89  98   9  63]
WARN: len(neuron_colors): 44 > n_cells: 41: restricting neuron_colors to the correct aclus, but if colors ever get off this is where it is happening!
merged_kwargs: {'name': 'spikeRasterOverviewWindowScatterPlotItem', 'pxMode': True, 'symbol': <PyQt5.QtGui.QPainterPath object at 0x7fb47739c270>, 'size': 2, 'pen': {'color': 'w', 'width': 1}, 'hoverable': True}
unit_sort_order: [36  1  9 15 25 29  8 32  6 30 31  3 34  4 19 35  7 11 12 24 28 17 21 22 18 20 23

In [ ]:
_out_rank_order_selected_spikes_only_event_raster_debugger.plots_data.on_update_active_scatterplot_kwargs(dict(pxMode=False))

In [ ]:
## Add the median spikes to the plots:
_active_plot_identifier = 'long_LR'

# Override the pen for the selected spots, the default renders them looking exactly like normal spikes which is no good:
for _active_plot_identifier, a_point_list in _out_rank_order_event_raster_debugger.plots_data.all_selected_spots_dict.items():
	# for a_point in _out_rank_order_event_raster_debugger.plots_data.all_selected_spots_dict[_active_plot_identifier]:
	for a_point in a_point_list:
		a_point['pen'] = pg.mkPen('#ffffff4e', width=1.5)
		a_point['brush'] = pg.mkBrush('#ffffff1f')
		
	a_scatter_plot = _out_rank_order_event_raster_debugger.plots.scatter_plots[_active_plot_identifier]
	# a_scatter_plot.updateSpots(_out_rank_order_event_raster_debugger.plots_data.all_selected_spots_dict[_active_plot_identifier])
	a_scatter_plot.addPoints(_out_rank_order_event_raster_debugger.plots_data.all_selected_spots_dict[_active_plot_identifier])

In [ ]:
_out_rank_order_event_raster_debugger.plots_data.all_selected_spots_dict

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, ExportFiletype

export_path = Path(r'C:\Users\pho\Desktop\2023-12-19 Exports').resolve()

_out_rank_order_event_raster_debugger.save_figure(export_path=export_path)


## Iterates through the epochs (via the slider) and saves out the images:


In [ ]:
all_save_paths = {}

for i in np.arange(0, _out_rank_order_event_raster_debugger.n_epochs, 5):
    _out_rank_order_event_raster_debugger.ui.ctrls_widget.setValue(i)
    # _out_rank_order_event_raster_debugger.on_update_epoch_IDX(an_epoch_idx=i)
    active_epoch_label = _out_rank_order_event_raster_debugger.active_epoch_label

    save_paths = []

    for a_decoder, a_plot in _out_rank_order_event_raster_debugger.root_plots_dict.items():
        curr_filename_prefix = f'Epoch{active_epoch_label}_{a_decoder}'
        # a_plot.setYRange(-0.5, float(self.max_n_neurons))
        out_path = export_path.joinpath(f'{curr_filename_prefix}_plot.png').resolve()
        export_pyqtgraph_plot(a_plot, savepath=out_path, background=pg.mkColor(0, 0, 0, 0))
        save_paths.append(out_path)

    all_save_paths[active_epoch_label] = save_paths
  
all_save_paths

In [ ]:
_out_rank_order_event_raster_debugger.active_epoch_IDX

In [ ]:
_out_rank_order_event_raster_debugger.active_epoch_result_df

In [ ]:
aclu_y_values_dict = {_active_plot_identifier:{int(aclu):new_sorted_raster.neuron_y_pos[aclu] for aclu in new_sorted_raster.neuron_IDs} for _active_plot_identifier, new_sorted_raster in _out_rank_order_event_raster_debugger.plots_data.seperate_new_sorted_rasters_dict.items()}
aclu_max_y_values_dict = {_active_plot_identifier:np.max(list({int(aclu):new_sorted_raster.neuron_y_pos[aclu] for aclu in new_sorted_raster.neuron_IDs}.values())) for _active_plot_identifier, new_sorted_raster in _out_rank_order_event_raster_debugger.plots_data.seperate_new_sorted_rasters_dict.items()} # {'long_LR': 51.48039215686274, 'long_RL': 53.5, 'short_LR': 51.48039215686274, 'short_RL': 53.5}
global_max_y_value = np.max(list(aclu_max_y_values_dict.values()))
global_max_y_value

In [ ]:
max_n_neurons = np.max([len(v) for v in _out_rank_order_event_raster_debugger.plots_data.unsorted_original_neuron_IDs_lists])
max_n_neurons

In [ ]:
_out_rank_order_event_raster_debugger.plots.all_separate_plots['long_LR']['root_plot']


root_plots_dict

In [ ]:
max_n_neurons

In [ ]:


all_separate_root_plots = {a_decoder_name:a_raster_setup_tuple.plots.root_plot for a_decoder_name, a_raster_setup_tuple in rasters_display_outputs.items()}


In [ ]:
_out_rank_order_selected_spikes_only_event_raster_debugger.ui.tableView

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import a_debug_callback_fn, debug_update_long_short_info_titles, debug_update_plot_titles

_out_LAPS_rank_order_event_raster_debugger = RankOrderRastersDebugger.init_rank_order_debugger(spikes_df, deepcopy(rank_order_results.LR_laps.epochs_df), track_templates, rank_order_results, None, None)
_out_LAPS_rank_order_event_raster_debugger.params.is_laps = True
_out_LAPS_rank_order_event_raster_debugger.on_idx_changed_callback_function_dict['debug_update_plot_titles_callback'] = debug_update_plot_titles

In [ ]:
rank_order_results.RL_ripple.selected_spikes_df

In [ ]:
# Scroll to the specified row
tableView.scrollTo(tableView.model().index(row_index, 0))

# Select the entire row
tableView.selectRow(row_index)

In [ ]:
_out_rank_order_event_raster_debugger.plots_data.LR_selected_spike_df #.LR_active_epochs_selected_spikes_fragile_linear_neuron_IDX_dict
# _out_rank_order_event_raster_debugger.plots_data.LR_neuron_id_to_new_IDX_map

In [ ]:
_obj = _out_rank_order_event_raster_debugger
## Add the spikes
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.LR_selected_spike_df), _active_plot_identifier = 'long_LR')
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.LR_selected_spike_df), _active_plot_identifier = 'short_LR')
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.RL_selected_spike_df), _active_plot_identifier = 'long_RL')
_obj.add_selected_spikes_df_points_to_scatter_plot(plots_data=_obj.plots_data, plots=_obj.plots, selected_spikes_df=deepcopy(_obj.plots_data.RL_selected_spike_df), _active_plot_identifier = 'short_RL')

In [ ]:
list(_out_rank_order_event_raster_debugger.plots.keys())
_out_rank_order_event_raster_debugger.plots['scatter_plots']['long_LR'].addPoints()



In [ ]:
list(_out_rank_order_event_raster_debugger.plots_data.keys())


In [ ]:
_out_rank_order_event_raster_debugger.plots_data['all_selected_spots_dict']


In [ ]:
_out_rank_order_event_raster_debugger.plots_data['seperate_all_spots_dict']

In [ ]:
_active_plot_identifier = 'long_LR'
new_sorted_raster = _out_rank_order_event_raster_debugger.plots_data['seperate_new_sorted_rasters_dict'][_active_plot_identifier]
# selected_spikes_df = new_sorted_raster.update_spikes_df_visualization_columns(spikes_df=selected_spikes_df)
# selected_spikes_df
new_sorted_raster.spikes_df

In [ ]:
## Add the source data (spikes_df) to the plot_data
plots_data.spikes_df = deepcopy(spikes_df)    
# Update the dataframe
plots_data.spikes_df = new_sorted_raster.update_spikes_df_visualization_columns(spikes_df=plots_data.spikes_df)
## Build the spots for the raster plot:
_out_rank_order_event_raster_debugger.plots_data['all_selected_spots_dict'][_active_plot_identifier], _out_rank_order_event_raster_debugger.plots_data.all_selected_scatterplot_tooltips_kwargs_dict[_active_plot_identifier] = new_sorted_raster.build_spikes_all_spots_from_df(spikes_df=plots_data.spikes_df, should_return_data_tooltips_kwargs=True, generate_debug_tuples=False)


In [ ]:
_out_rank_order_event_raster_debugger.plots_data.spikes_df

In [ ]:
_out_rank_order_event_raster_debugger.global_spikes_df

In [ ]:
add_selected_spikes_df_points_to_scatter_plot

## Connects TemplatesDebugger to RasterDebugger

In [25]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.ContainerBased.RankOrderRastersDebugger import a_debug_callback_fn, debug_update_long_short_info_titles, debug_update_plot_titles


# active_plotter = _out_rank_order_event_raster_debugger
active_plotter = _out_rank_order_selected_spikes_only_event_raster_debugger

# _out_directional_template_pfs_debugger = None

def plot_attached_directional_templates_pf_debugger(curr_active_pipeline, rank_order_raster_debugger: RankOrderRastersDebugger):
    """ builds a _display_directional_template_debugger, attaches it to the provided rank_order_event_raster_debugger so it's updated on its callback, and then returns what it created. 
    
    """
    curr_active_pipeline.reload_default_display_functions()
    # epoch_active_aclus = np.array([9,  26,  31,  39,  40,  43,  47,  52,  53,  54,  60,  61,  65,  68,  72,  75,  77,  78,  81,  82,  84,  85,  90,  92,  93,  98, 102]) # some test indicies
    epoch_active_aclus = None
    _out_directional_template_pfs_debugger = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_template_debugger, included_any_context_neuron_ids=epoch_active_aclus, figure_name=f'<Controlled by RankOrderRastersDebugger>')

    def debug_update_paired_directional_template_pfs_debugger(a_plotter, an_idx: int):
        """ captures: _out_directional_template_pfs_debugger, """
        epoch_active_aclus = deepcopy(a_plotter.get_epoch_active_aclus())
        # update the displayed cells:
        directional_template_pfs_debugger_on_update_callback = _out_directional_template_pfs_debugger.get('ui').on_update_callback
        directional_template_pfs_debugger_on_update_callback(epoch_active_aclus)
    
    rank_order_raster_debugger.on_idx_changed_callback_function_dict['debug_update_paired_directional_template_pfs_debugger'] = debug_update_paired_directional_template_pfs_debugger
    
    return _out_directional_template_pfs_debugger, debug_update_paired_directional_template_pfs_debugger


_out_directional_template_pfs_debugger, debug_update_paired_directional_template_pfs_debugger = plot_attached_directional_templates_pf_debugger(curr_active_pipeline, rank_order_raster_debugger=active_plotter)

active_plotter.on_idx_changed_callback_function_dict['a_debug_callback'] = a_debug_callback_fn
active_plotter.on_idx_changed_callback_function_dict['debug_update_plot_titles_callback'] = debug_update_plot_titles
# active_plotter.on_idx_changed_callback_function_dict['debug_update_paired_directional_template_pfs_debugger'] = debug_update_paired_directional_template_pfs_debugger
# active_plotter.on_idx_changed_callback_function_dict['debug_update_long_short_info_titles'] = debug_update_long_short_info_titles
# active_plotter.on_idx_changed_callback_function_dict['debug_plot_epoch_label_row'] = debug_plot_epoch_label_row

# active_plotter.on_update_epoch_IDX(11)

PhoDockAreaContainingWindow.GlobalConnectionManagerAccessingMixin_on_setup()
PhoDockAreaContainingWindow.try_register_any_control_widgets()
	flat_widgets_list contains 0 items
using overriden dock location.


_tip_fn(...): data_string: t: 188.83924539433792|aclu: 41|neuron_IDX: 5|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 187.58897275442723|aclu: 41|neuron_IDX: 5|visualization_raster_y_location: nan
_tip_fn(...): data_string: t: 182.40067461086437|aclu: 41|neuron_IDX: 5|visualization_raster_y_location: nan
emitChanged(): self.val: 9
valueChanged(new_val: 9)
an_epoch: EpochTuple(Index=11, start=56.1137376697734, stop=56.206788502051495, label=11, duration=0.09305083227809519, LR_Long_ActuallyIncludedAclus=array([ 41,  48,  53,  55,  59,  62,  86,  88, 108]), LR_Long_rel_num_cells=9, RL_Long_ActuallyIncludedAclus=array([ 41,  48,  53,  55,  59,  62,  86,  88, 108]), RL_Long_rel_num_cells=9, LR_Short_ActuallyIncludedAclus=array([ 41,  48,  53,  55,  59,  62,  86,  88, 108]), LR_Short_rel_num_cells=9, RL_Short_ActuallyIncludedAclus=array([ 41,  48,  53,  55,  59,  62,  86,  88, 108]), RL_Short_rel_num_cells=9, LR_evidence=6.33058753936408, RL_evidence=2.66941246063592, L

In [ ]:
active_epochs_df = deepcopy(ripple_result_tuple.active_epochs)
active_epochs_df

In [ ]:
active_epochs_df = deepcopy(rank_order_results.LR_ripple.epochs_df)
active_epochs_df

In [ ]:
print(list(rank_order_raster_debugger.global_spikes_df.columns)) # ['start', 'stop', 'label', 'duration', 'end', 'LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson', 'LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman', 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']



In [ ]:
print(list(active_selected_spikes_df.columns)) # ['start', 'stop', 'label', 'duration', 'end', 'LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson', 'LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman', 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']



In [ ]:
print(list(active_epochs_df.columns)) # ['start', 'stop', 'label', 'duration', 'end', 'LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson', 'LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman', 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']


a_row, a_label = get_epoch_label_row(188)
a_row



In [ ]:
a_row = list(a_row.itertuples())[0]

In [ ]:
list(rank_order_raster_debugger.plots_data.keys())


In [ ]:
debug_update_plot_titles(rank_order_raster_debugger, an_idx=20)

In [ ]:
a_debug_callback_fn(rank_order_raster_debugger, an_idx=20)

In [ ]:
list(rank_order_raster_debugger.plots.root_plots.keys())


In [ ]:
directional_likelihoods_tuple: DirectionalRankOrderLikelihoods = ripple_result_tuple.directional_likelihoods_tuple
directional_likelihoods_tuple.long_best_direction_indices
ripple_result_tuple.rank_order_z_score_df

In [ ]:
ripple_result_tuple.rank_order_z_score_df.label

### Independent DirectionalTemplatePFsDebugger for testing

In [ ]:
# _display_directional_laps_overview:
curr_active_pipeline.prepare_for_display()
curr_active_pipeline.reload_default_display_functions()
session_description: str = curr_active_pipeline.get_session_context().get_description()
print(f'session_description: {session_description}')
pg.setConfigOptions(imageAxisOrder='row-major')
# _out = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_laps_overview)
_out_all_cells_directional_template_pfs_debugger = curr_active_pipeline.display(DirectionalPlacefieldGlobalDisplayFunctions._display_directional_template_debugger, included_any_context_neuron_ids=None, figure_name='All Cells (Independent)')

In [ ]:
plots = _out_all_cells_directional_template_pfs_debugger['plots']
a_plot_widget, an_image_item = plots['pf1D_heatmaps']['long_LR'] # (pyqtgraph.widgets.PlotWidget.PlotWidget, pyqtgraph.graphicsItems.ImageItem.ImageItem)
a_view_box = an_image_item.getViewBox()
a_plot_widget


In [ ]:
rois = []
rois.append(pg.RectROI([20, 20], [20, 20], pen=(0,9)))


In [ ]:

# Function to handle hover event
def hoverEvent(pos):
    print(f'hoverEvent')
    index = int(pos.x())
    region.setRegion([index, index + 1])  # Highlight the entire row

# Add LinearRegionItem for highlighting
region = pg.LinearRegionItem(brush='#FFFFFF', hoverBrush='#FFFFFF', movable=False, clipItem=an_image_item)
region.setZValue(10)

a_plot_widget.addItem(region, ignoreBounds=True)

# Connect hover event
a_plot_widget.scene().sigMouseMoved.connect(hoverEvent)

region.setRegion([0.0, 10.0 + 1])

In [ ]:
# Set spike emphasis on debug rasters?

# Get debug rasters to match the colors in template viewer?

from numpy import ma

def _perform_compute_directional_likelihoods_tuple_methods(active_epochs, active_LR_long_z_score=None, active_RL_long_z_score=None, active_LR_short_z_score=None, active_RL_short_z_score=None) -> DirectionalRankOrderLikelihoods:
	""" Used to switch between three different temporary methods of determining the proper direction.
	captures curr_active_pipeline, global_epoch_name """
	# ALT_METHOD : main `_compute_best_direction_likelihoods`
	# #TODO 2023-12-12 11:56: - [ ] Should these be saved to the ripple result?
	# spikes_df = deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].spikes_df)
	# # Unpacking for `(long_LR_name, long_RL_name, short_LR_name, short_RL_name)`
	# (long_LR_results, long_RL_results, short_LR_results, short_RL_results) = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in (long_LR_name, long_RL_name, short_LR_name, short_RL_name)]
	# return cls._compute_best_direction_likelihoods(active_epochs, long_LR_results, long_RL_results, short_LR_results, short_RL_results, spikes_df, decoding_time_bin_size)

	## ALT 2023-12-12 - APPROXIMATE best-direction without using the decoder as a workaround since `cls._compute_best_direction_likelihoods` isn't working yet:
	print(f'WARNING: approximating best-direction without using the decoder as a workaround.')

	# ALT_METHOD: mean_determined_epoch_dir: `mean_determined_epoch_dir`
	# mean_LR_z_scores = ((np.abs(rank_order_results.LR_ripple.long_z_score) + np.abs(rank_order_results.LR_ripple.short_z_score)) / 2.0)
	# mean_RL_z_scores = ((np.abs(rank_order_results.RL_ripple.long_z_score) + np.abs(rank_order_results.RL_ripple.short_z_score)) / 2.0)
	# mean_determined_epoch_dir = (mean_LR_z_scores >= mean_RL_z_scores).astype(int)
	# long_best_direction_indicies = mean_determined_epoch_dir.copy()
	# short_best_direction_indicies = mean_determined_epoch_dir.copy()

	# ALT_METHOD: best max method:
	# filtered_active_epochs, (active_LR_long_z_score, active_RL_long_z_score, active_LR_short_z_score, active_RL_short_z_score) = rank_order_results.get_aligned_events(active_epochs)
	long_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_long_z_score), np.abs(active_RL_long_z_score)]), axis=0).astype(int)
	short_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_short_z_score), np.abs(active_RL_short_z_score)]), axis=0).astype(int)

	## build the new directional_likelihods_tuple:
	#TODO 2023-12-12 13:50: - [ ] Add `filtered_active_epochs` to the DirectionalRankOrderLikelihoods?
	return DirectionalRankOrderLikelihoods(long_relative_direction_likelihoods=None, short_relative_direction_likelihoods=None, long_best_direction_indices=long_best_direction_indicies, short_best_direction_indices=short_best_direction_indicies)


## Post-process Z-scores with their most likely directions:
directional_laps_results: DirectionalLapsResult = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
rank_order_results: RankOrderComputationsContainer = curr_active_pipeline.global_computation_results.computed_data['RankOrder']

ripple_combined_epoch_stats_df = deepcopy(rank_order_results.ripple_combined_epoch_stats_df)
active_replay_epochs = deepcopy(rank_order_results.LR_ripple.epochs_df)

_traditional_arg_dict_key_names = ('active_LR_long_z_score', 'active_RL_long_z_score', 'active_LR_short_z_score', 'active_RL_short_z_score')
_traditional_arg_dict_values_tuple = (ripple_combined_epoch_stats_df.LR_Long_spearman_Z, ripple_combined_epoch_stats_df.RL_Long_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z)
active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score = ripple_combined_epoch_stats_df.LR_Long_spearman_Z, ripple_combined_epoch_stats_df.RL_Long_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z, ripple_combined_epoch_stats_df.LR_Short_spearman_Z


_arg_dict_from_ripple_combined_epoch_stats_df = dict(zip(_traditional_arg_dict_key_names, _traditional_arg_dict_values_tuple))
ripple_directional_likelihoods_tuple: DirectionalRankOrderLikelihoods = _perform_compute_directional_likelihoods_tuple_methods(None, **_arg_dict_from_ripple_combined_epoch_stats_df)
long_relative_direction_likelihoods, short_relative_direction_likelihoods, long_best_direction_indicies, short_best_direction_indicies = ripple_directional_likelihoods_tuple


ripple_evts_long_best_dir_z_score_values = np.where(long_best_direction_indicies, active_LR_ripple_long_z_score, active_RL_ripple_long_z_score)
ripple_evts_short_best_dir_z_score_values = np.where(short_best_direction_indicies, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score)
ripple_evts_long_short_best_dir_z_score_diff_values = ripple_evts_long_best_dir_z_score_values - ripple_evts_short_best_dir_z_score_values

_LR_INDEX = 0
_RL_INDEX = 1
ripple_masked_z_score_values_list: List[ma.masked_array] = [ma.masked_array(x, mask=np.logical_not(a_mask)) for x, a_mask in zip((active_LR_ripple_long_z_score, active_RL_ripple_long_z_score, active_LR_ripple_short_z_score, active_RL_ripple_short_z_score),
																									((long_best_direction_indicies == _LR_INDEX), (long_best_direction_indicies == _RL_INDEX), (short_best_direction_indicies == _LR_INDEX), (short_best_direction_indicies == _RL_INDEX)))]


# outputs: ripple_evts_long_short_best_dir_z_score_diff_values
ripple_result_tuple: DirectionalRankOrderResult = DirectionalRankOrderResult(active_replay_epochs, long_best_dir_z_score_values=ripple_evts_long_best_dir_z_score_values, short_best_dir_z_score_values=ripple_evts_short_best_dir_z_score_values,
																			long_short_best_dir_z_score_diff_values=ripple_evts_long_short_best_dir_z_score_diff_values, directional_likelihoods_tuple=ripple_directional_likelihoods_tuple,
																			masked_z_score_values_list=ripple_masked_z_score_values_list, rank_order_z_score_df=None)



In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses

## Main
ripple_result_tuple, laps_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.003)
ripple_result_tuple

In [ ]:
laps_result_tuple

In [ ]:
curr_active_pipeline.global_computation_results.computed_data['RankOrder'].ripple_most_likely_result_tuple, curr_active_pipeline.global_computation_results.computed_data['RankOrder'].laps_most_likely_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.006) # 6ms bins

# Plot the z-scores differences and their raw-values

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import plot_rank_order_epoch_inst_fr_result_tuples

ripple_outputs = plot_rank_order_epoch_inst_fr_result_tuples(curr_active_pipeline, ripple_result_tuple, 'Ripple')

In [ ]:
# Usage of the function for Lap
lap_outputs = plot_rank_order_epoch_inst_fr_result_tuples(curr_active_pipeline, laps_result_tuple, 'Lap')

In [ ]:
rank_order_results.

In [ ]:
an_idx: int = 154
# a_label = lookup_label_from_index(an_idx)
# a_label

In [ ]:
active_selected_spikes_df[active_selected_spikes_df['Probe_Epoch_id'] == an_idx]['aclu'].count()

In [ ]:
active_selected_spikes_df[active_selected_spikes_df['Probe_Epoch_id'] == an_idx].plot.scatter(x='t_rel_seconds', y='LR_Long_pf_peak_x')

In [ ]:
## filter events by (|z| > 1.0)
ripple_result_tuple.active_epochs
ripple_result_tuple.long_best_dir_z_score_values

In [ ]:
# ripple_result_tuple.directional_likelihoods_tuple
ripple_result_tuple.directional_likelihoods_tuple.long_best_direction_indices[105] # 1, 

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.short_best_direction_indices[105]

In [ ]:
ripple_result_tuple.long_best_dir_z_score_values[105]

In [ ]:
ripple_result_tuple.short_best_dir_z_score_values[105]

In [ ]:
ripple_result_tuple.active_epochs

In [ ]:
np.shape(ripple_result_tuple.long_best_dir_z_score_values)

In [ ]:
np.shape(ripple_result_tuple.short_best_dir_z_score_values)

In [ ]:
global_replays.n_epochs

In [ ]:
global_replays

In [ ]:
ratemap = long_pf1D.ratemap
ratemap.tuning_curve_unsmoothed_peak_firing_rates

In [ ]:
type(rank_order_results)

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import DirectionalRankOrderResult

# type(ripple_result_tuple)
ripple_result_tuple.plot_histograms()

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.long_relative_direction_likelihoods.shape

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.short_relative_direction_likelihoods.shape

In [ ]:
ripple_result_tuple.directional_likelihoods_tuple.long_best_direction_indices

In [ ]:
ripple_result_tuple.short_best_dir_z_score_values

In [ ]:
# 2023-11-16_LapsRankOrderHistogram Figure:
pd.DataFrame({'long_z_scores': laps_result_tuple.long_best_dir_z_score_values, 'short_z_scores': laps_result_tuple.short_best_dir_z_score_values}).hist()

# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:


In [ ]:
curr_active_pipeline.prepare_for_display()

In [ ]:
# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
# active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.display('_display_spike_rasters_pyqtplot_2D', 'maze_any') # 'maze_any'


In [ ]:
active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()


In [ ]:
spike_raster_window

In [ ]:
spikes_window = spike_raster_window.spikes_window # SpikesDataframeWindow

bottomPlaybackControlBarWidget = spike_raster_window.ui.bottomPlaybackControlBarWidget # Spike3DRasterBottomPlaybackControlBar 

doubleSpinBox_ActiveWindowStartTime = bottomPlaybackControlBarWidget.ui.doubleSpinBox_ActiveWindowStartTime
doubleSpinBox_ActiveWindowEndTime = bottomPlaybackControlBarWidget.ui.doubleSpinBox_ActiveWindowEndTime


# spikes_window.timeWindow.start
# spikes_window.active_window_start_time
# spikes_window.update_window_start_end(451.8908457518555, 451.9895490613999) ## Works but does not trigger refresh/update of the window. The changes are reflected as soon as you try to scroll at all though.
# spikes_window.active_window_end_time

print(f'spikes_window.active_window_start_time: {spikes_window.active_window_start_time}, spikes_window.active_window_end_time: {spikes_window.active_window_end_time}')
# need to block signals:
# doubleSpinBox_ActiveWindowStartTime.blockSignals(True)
# doubleSpinBox_ActiveWindowEndTime.blockSignals(True)
doubleSpinBox_ActiveWindowStartTime.setValue(spikes_window.active_window_start_time)
doubleSpinBox_ActiveWindowEndTime.setValue(spikes_window.active_window_end_time)
# doubleSpinBox_ActiveWindowStartTime.blockSignals(False) # unblock the signals when done
# doubleSpinBox_ActiveWindowEndTime.blockSignals(False)


# @pyqtExceptionPrintingSlot(float, float)
def on_active_window_changed(start_t, end_t, _obj):
	# need to block signals:
	# doubleSpinBox_ActiveWindowStartTime.blockSignals(True)
	# doubleSpinBox_ActiveWindowEndTime.blockSignals(True)
	if start_t is not None:
		doubleSpinBox_ActiveWindowStartTime.setValue(start_t)
	if end_t is not None:
		doubleSpinBox_ActiveWindowEndTime.setValue(end_t)
	# doubleSpinBox_ActiveWindowStartTime.blockSignals(False) # unblock the signals when done
	# doubleSpinBox_ActiveWindowEndTime.blockSignals(False)

curr_window_ctrls_connection = spikes_window.windowed_data_window_updated_signal.connect(on_active_window_changed)


In [ ]:
doubleSpinBox_ActiveWindowStartTime.setReadOnly(True)
doubleSpinBox_ActiveWindowEndTime.setReadOnly(True)

spikes_window.on_window_changed.connect(

In [ ]:
doubleSpinBox_ActiveWindowStartTime.setVisible(False)
bottomPlaybackControlBarWidget.setVisible(False)

In [ ]:
# global_epoch_context
curr_active_pipeline.reload_default_display_functions()
# curr_active_pipeline.prepare_for_display()
curr_active_pipeline.clear_display_outputs()


In [ ]:
# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper
import pyphoplacecellanalysis.External.pyqtgraph as pg # Used to get the app for TopLevelWindowHelper.top_level_windows
## For searching with `TopLevelWindowHelper.all_widgets(...)`:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

found_spike_raster_windows = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=Spike3DRasterWindowWidget)

if len(found_spike_raster_windows) < 1:
	# no existing spike_raster_windows. Make a new one
	print(f'no existing SpikeRasterWindow. Creating a new one.')
	# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
	# active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

	active_2d_plot, active_3d_plot, spike_raster_window = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

else:
	print(f'found {len(found_spike_raster_windows)} existing Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...). Will use the most recent.')
	# assert len(found_spike_raster_windows) == 1, f"found {len(found_spike_raster_windows)} Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...) but require exactly one."
	# Get the most recent existing one and reuse that:
	spike_raster_window = found_spike_raster_windows[0]


# Extras:
active_2d_plot = spike_raster_window.spike_raster_plt_2d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
active_3d_plot = spike_raster_window.spike_raster_plt_3d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
main_graphics_layout_widget = active_2d_plot.ui.main_graphics_layout_widget # GraphicsLayoutWidget
main_plot_widget = active_2d_plot.plots.main_plot_widget # PlotItem
background_static_scroll_plot_widget = active_2d_plot.plots.background_static_scroll_window_plot # PlotItem

In [ ]:
spike_raster_window.isVisible() # False
# spike_raster_window.show()
spike_raster_window.close()

In [ ]:
spike_raster_window.connection_man.active_connections

In [ ]:
found_any_window = TopLevelWindowHelper.top_level_windows(pg.mkQApp())
found_any_window

In [ ]:
# print windows:
[print_widget_hierarchy(v) for k, v in found_any_window.items()]

In [ ]:
_display_out = curr_active_pipeline.last_added_display_output


In [ ]:
ipspikesDataExplorer = _display_out['ipspikesDataExplorer']
pActiveSpikesBehaviorPlotter = _display_out['plotter']


In [ ]:
 = curr_active_pipeline.last_added_display_output
_display_out


In [ ]:
ipspikesDataExplorer = self._display_output['ipspikesDataExplorer']

In [ ]:
### Adjusting Spike Emphasis:
#### Usage Examples:
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState
from neuropy.core.neuron_identities import NeuronType


In [ ]:

## Example 1: De-emphasize spikes excluded from the placefield calculations:
is_spike_included_in_pf = np.isin(spike_raster_window.spike_raster_plt_2d.spikes_df.index, active_pf_2D.filtered_spikes_df.index)
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not(is_spike_included_in_pf), SpikeEmphasisState.Deemphasized)

## Example 2: De-emphasize spikes that don't have their 'aclu' from a given set of indicies:
is_spike_included = spike_raster_window.spike_raster_plt_2d.spikes_df.aclu.to_numpy() == 2
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not(is_spike_included), SpikeEmphasisState.Deemphasized)

## Example 3: De-emphasize all spikes 
active_2d_plot.update_spike_emphasis(new_emphasis_state=SpikeEmphasisState.Deemphasized)

## Example 4: Hide all spikes entirely
active_2d_plot.update_spike_emphasis(new_emphasis_state=SpikeEmphasisState.Hidden)


In [ ]:
## Setup: Hide all non-pyramidal spikes entirely
spikes_df = spike_raster_window.spikes_df
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not((spikes_df.neuron_type == NeuronType.from_string('pyr'))), SpikeEmphasisState.Hidden)

In [ ]:
spikes_window = spike_raster_window.spikes_window # SpikesDataframeWindow
# spikes_window.update_window_start_end(451.8908457518555, 451.9895490613999) ## Works but does not trigger refresh/update of the window. The changes are reflected as soon as you try to scroll at all though.


In [ ]:
# 20*60.0 + 50.0 +  0.218 = 1250.218

spikes_window.update_window_start_end(1250.218, (1250.218 + 3.0))



In [ ]:
spikes_window.window_duration # Prints the current window's duration. The win. dur. label control in the left bar is not updated.

desired_window_fraction: float = 0.1 # 10% of the window is the default jump size
relevant_jump_duration: float = spikes_window.window_duration * desired_window_fraction
relevant_jump_duration


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.mixins.spikes_mixins import SpikeRenderingPyVistaMixin
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer
# from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

found_windows_of_type = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=InteractivePlaceCellDataExplorer)
found_windows_of_type
TopLevelWindowHelper.top_level_windows(pg.mkQApp(), only_visible=True)

In [ ]:
(451.8908457518555, 451.9895490613999)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import BatchPhoJonathanFiguresHelper

fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True, disable_top_row=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}

# PhoKamran2023Paper Results

In [ ]:
pg.setConfigOptions(background='white', foreground='black') # black on white background (more traditional) color scheme

## Figure 1) pf1D Ratemaps, Active set, etc

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters

curr_active_pipeline.prepare_for_display()
pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics, fig_1c_figures_out_dict = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
# rdf = jonathan_firing_rate_analysis_result.rdf.rdf
# rdf
# ==================================================================================================================== #
# Fig 1c) 2023-07-14 - LxC and SxC PhoJonathanSession plots                                                            #
# ==================================================================================================================== #
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import BatchPhoJonathanFiguresHelper

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis.neuron_replay_stats_df, curr_jonathan_firing_rate_analysis.rdf.rdf, curr_jonathan_firing_rate_analysis.rdf.aclu_to_idx, curr_jonathan_firing_rate_analysis.irdf.irdf

fig_1c_figures_out_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True, disable_top_row=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}


## Figure 2) `PaperFigureTwo`: LxC/SxC Analyses
Note: this fails when SxC or LxC are empty for this session (as it's not meaningful to produce a comparison bar plot). In this case, aggregate across multiple sessions.

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline)
_out_fig_2.display()

## Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [ ]:
from neuropy.utils.matplotlib_helpers import FormattedFigureText
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies
# curr_active_pipeline.reload_default_display_functions()

_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=True)

##  All Programmatic Plots

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots


batch_perform_all_plots(curr_active_pipeline, enable_neptune=False, neptuner=None)

# Simple ActiveSet Directional Determination

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderResult
from pyphocorehelpers.indexing_helpers import bidirectional_setdiff1d

# rank_order_results.LR_ripple.extra_info_dict

# active_epochs_df


In [ ]:
bidirectional_setdiff1d(rank_order_results.LR_ripple.epochs_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy())

In [ ]:
# spikes_df.flat_spike_idx
# np.setdiff1d(rank_order_results.LR_ripple.spikes_df.flat_spike_idx.to_numpy(), rank_order_results.RL_ripple.spikes_df.flat_spike_idx.to_numpy())
bidirectional_setdiff1d(rank_order_results.LR_ripple.spikes_df.flat_spike_idx.to_numpy(), rank_order_results.RL_ripple.spikes_df.flat_spike_idx.to_numpy())

In [ ]:
display(bidirectional_setdiff1d(rank_order_results.LR_ripple.spikes_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy()))
display(np.setdiff1d(rank_order_results.LR_ripple.spikes_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy()))

In [ ]:
rank_order_results.RL_ripple.epochs_df

In [62]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderComputationsContainer
# from PendingNotebookCode import epoch_directionality_active_set_evidence
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import RankOrderAnalyses
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.RankOrderComputations import DirectionalRankOrderLikelihoods

# active_epochs_df.label.astype(label_column_type).map(lambda x: rank_order_results.LR_ripple.epoch_template_active_aclus[x])

# ['LR_Long_ActuallyIncludedAclus', 'RL_Long_ActuallyIncludedAclus', 'LR_Short_ActuallyIncludedAclus', 'RL_Short_ActuallyIncludedAclus']

# rank_order_results.LR_laps.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.LR_laps.epochs_df, is_laps=True)
# rank_order_results.RL_laps.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.RL_laps.epochs_df, is_laps=True)

# rank_order_results.LR_ripple.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.LR_ripple.epochs_df, is_laps=False)
# rank_order_results.RL_ripple.epochs_df = add_active_aclus_info(rank_order_results, active_epochs_df=rank_order_results.RL_ripple.epochs_df, is_laps=False)



In [63]:

rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple = None, None

In [69]:
rank_order_results.adding_active_aclus_info()
rank_order_results.ripple_most_likely_result_tuple, rank_order_results.laps_most_likely_result_tuple = RankOrderAnalyses.most_likely_directional_rank_order_shuffling(curr_active_pipeline, decoding_time_bin_size=0.006) # 6ms bins
        
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.long_relative_direction_likelihoods

array([0.524619, 0.532065, 0.526262, 0.531576, 0.543372, 0.524335, 0.543576, 0.537815, 0.520777, 0.536865, 0.526262, 0.534945, 0.537222, 0.539937, 0.51875, 0.530008, 0.522368, 0.548632, 0.522144, 0.54591, 0.51384, 0.551665, 0.500207, 0.539937, 0.543366, 0.550332, 0.533054, 0.551254, 0.532536, 0.536865, 0.532536, 0.539937, 0.512283, 0.526262, 0.518631, 0.535646, 0.522234, 0.528452, 0.525323, 0.527709, 0.525638, 0.526262, 0.51875, 0.538496, 0.5265, 0.551665, 0.526262, 0.558667, 0.526262, 0.523075, 0.532536, 0.554471, 0.526262, 0.535494, 0.515592, 0.526262, 0.515592, 0.532872, 0.518402, 0.560276, 0.506059, 0.554471, 0.503252, 0.553954, 0.532848, 0.54386, 0.549881, 0.535646, 0.522252, 0.539764, 0.526262, 0.526262, 0.539743, 0.526262, 0.515115, 0.543636, 0.51875, 0.543981, 0.547612, 0.562844])

In [67]:
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.short_best_direction_indices


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [ ]:


long_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_long_z_score), np.abs(active_RL_long_z_score)]), axis=0).astype(int)
short_best_direction_indicies = np.argmax(np.vstack([np.abs(active_LR_short_z_score), np.abs(active_RL_short_z_score)]), axis=0).astype(int)

In [50]:
recovered_lap_epochs = rank_order_results.laps_most_likely_result_tuple.active_epochs
estimated_directions = deepcopy(rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple.long_best_direction_indices)
assert len(recovered_lap_epochs) == len(estimated_directions), f"recovered_lap_epochs should be same length as estimated_directions"
estimated_directions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
recovered_lap_epochs['estimated_direction_index'] = estimated_directions
recovered_lap_epochs

In [42]:
recovered_lap_epochs

,start,stop,label,duration,LR_Long_ActuallyIncludedAclus,LR_Long_rel_num_cells,RL_Long_ActuallyIncludedAclus,RL_Long_rel_num_cells,LR_Short_ActuallyIncludedAclus,LR_Short_rel_num_cells,RL_Short_ActuallyIncludedAclus,RL_Short_rel_num_cells,LR_evidence,RL_evidence,LR_product_evidence,RL_product_evidence,normed_LR_evidence,normed_RL_evidence,normed_product_LR_evidence,normed_product_RL_evidence
0,5.664919,17.359802,0,11.694883,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,17.312439,15.687561,7.051410e-12,5.073033e-13,0.524619,0.475381,0.932885,0.067115
1,31.961164,39.764808,1,7.803644,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,18.090217,15.909783,5.484430e-12,1.127341e-13,0.532065,0.467935,0.979859,0.020141
2,135.834493,144.148150,2,8.313657,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,18.945425,17.054575,9.609542e-13,3.607567e-14,0.526262,0.473738,0.963817,0.036183
3,161.553817,167.318145,3,5.764329,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,18.605172,16.394828,2.824237e-12,5.468104e-14,0.531576,0.468424,0.981006,0.018994
4,234.470126,239.718943,4,5.248817,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,15.757783,13.242217,4.468796e-10,4.281312e-12,0.543372,0.456628,0.990510,0.009490
5,255.250813,262.688459,5,7.437646,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,17.827394,16.172606,3.631162e-12,2.460647e-13,0.524335,0.475665,0.936536,0.063464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1999.045622,2002.816193,74,3.770571,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,13.392977,12.607023,5.891403e-10,2.661151e-10,0.515115,0.484885,0.688847,0.311153
75,2003.032708,2008.278298,75,5.245591,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,18.483616,15.516384,2.019684e-11,6.158665e-14,0.543636,0.456364,0.996960,0.003040
76,2019.176367,2024.898701,76,5.722334,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,16.081247,14.918753,1.956409e-11,3.572323e-12,0.518750,0.481250,0.845597,0.154403


In [37]:
rank_order_results.LR_laps.epochs_df

,start,stop,label,duration,LR_Long_ActuallyIncludedAclus,LR_Long_rel_num_cells,RL_Long_ActuallyIncludedAclus,RL_Long_rel_num_cells,LR_Short_ActuallyIncludedAclus,LR_Short_rel_num_cells,RL_Short_ActuallyIncludedAclus,RL_Short_rel_num_cells,LR_evidence,RL_evidence,LR_product_evidence,RL_product_evidence,normed_LR_evidence,normed_RL_evidence,normed_product_LR_evidence,normed_product_RL_evidence
0,5.664919,17.359802,0,11.694883,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,17.312439,15.687561,7.051410e-12,5.073033e-13,0.524619,0.475381,0.932885,0.067115
1,31.961164,39.764808,1,7.803644,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,18.090217,15.909783,5.484430e-12,1.127341e-13,0.532065,0.467935,0.979859,0.020141
2,135.834493,144.148150,2,8.313657,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,18.945425,17.054575,9.609542e-13,3.607567e-14,0.526262,0.473738,0.963817,0.036183
3,161.553817,167.318145,3,5.764329,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,18.605172,16.394828,2.824237e-12,5.468104e-14,0.531576,0.468424,0.981006,0.018994
4,234.470126,239.718943,4,5.248817,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,15.757783,13.242217,4.468796e-10,4.281312e-12,0.543372,0.456628,0.990510,0.009490
5,255.250813,262.688459,5,7.437646,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,17.827394,16.172606,3.631162e-12,2.460647e-13,0.524335,0.475665,0.936536,0.063464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1999.045622,2002.816193,74,3.770571,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,13.392977,12.607023,5.891403e-10,2.661151e-10,0.515115,0.484885,0.688847,0.311153
75,2003.032708,2008.278298,75,5.245591,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,18.483616,15.516384,2.019684e-11,6.158665e-14,0.543636,0.456364,0.996960,0.003040
76,2019.176367,2024.898701,76,5.722334,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,16.081247,14.918753,1.956409e-11,3.572323e-12,0.518750,0.481250,0.845597,0.154403


In [40]:
# global_laps
global_any_laps_epochs_obj.to_dataframe()['lap_dir']

0     0
1     1
2     0
3     1
4     0
5     1
     ..
74    0
75    1
76    0
77    1
78    0
79    1
Name: lap_dir, Length: 80, dtype: int64

In [35]:
rank_order_results.laps_most_likely_result_tuple.plot_histograms()


MatplotlibRenderPlots({'name': 'plot_histogram_figure', 'context': None, 'figures': [<Figure size 640x480 with 3 Axes>], 'axes': {'long_short_best_z_score_diff': <Axes: label='long_short_best_z_score_diff', title={'center': 'long_short_best_z_score_diff'}>, 'long_best_z_scores': <Axes: label='long_best_z_scores', title={'center': 'long_best_z_scores'}>, 'short_best_z_scores': <Axes: label='short_best_z_scores', title={'center': 'short_best_z_scores'}>}})

In [17]:
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple

DirectionalRankOrderLikelihoods(long_relative_direction_likelihoods=None, short_relative_direction_likelihoods=None, long_best_direction_indices=array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1]), short_best_direction_indices=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [22]:
rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple
, 


DirectionalRankOrderLikelihoods(long_relative_direction_likelihoods=0     0.559657
1     0.573001
2     0.564693
3     0.571294
4     0.584085
5     0.558303
        ...   
74    0.524252
75    0.584922
76    0.543603
77    0.585278
78    0.599753
79    0.620346
Name: normed_LR_evidence, Length: 80, dtype: float64, short_relative_direction_likelihoods=0     0.440343
1     0.426999
2     0.435307
3     0.428706
4     0.415915
5     0.441697
        ...   
74    0.475748
75    0.415078
76    0.456397
77    0.414722
78    0.400247
79    0.379654
Name: normed_RL_evidence, Length: 80, dtype: float64, long_best_direction_indices=0     True
1     True
2     True
3     True
4     True
5     True
      ... 
74    True
75    True
76    True
77    True
78    True
79    True
Length: 80, dtype: bool, short_best_direction_indices=0     True
1     True
2     True
3     True
4     True
5     True
      ... 
74    True
75    True
76    True
77    True
78    True
79    True
Length: 80, dtype: bool)

In [13]:

# Requires `active_epochs_df` coming in:

# recieves lists of identities (such as cell aclus) and a function that returns a sortable value for each identity:
directional_laps_results = curr_active_pipeline.global_computation_results.computed_data['DirectionalLaps']
track_templates: TrackTemplates = directional_laps_results.get_templates(minimum_inclusion_fr_Hz=rank_order_results.minimum_inclusion_fr_Hz) # non-shared-only
decoders_dict = track_templates.get_decoders_dict() # decoders_dict = {'long_LR': track_templates.long_LR_decoder, 'long_RL': track_templates.long_RL_decoder, 'short_
# LR': track_templates.short_LR_decoder, 'short_RL': track_templates.short_RL_decoder, }

# global_replays = TimeColumnAliasesProtocol.renaming_synonym_columns_if_needed(deepcopy(curr_active_pipeline.filtered_sessions[global_epoch_name].replay))
# active_replay_epochs, active_epochs_df, active_selected_spikes_df = combine_rank_order_results(rank_order_results, global_replays, track_templates=track_templates)

# 
# ['start', 'stop', 'label', 'duration', 'LR_Long_spearman', 'RL_Long_spearman', 'LR_Short_spearman', 'RL_Short_spearman', 'LR_Long_pearson', 'RL_Long_pearson', 'LR_Short_pearson', 'RL_Short_pearson', 'LR_Long_Old_Spearman', 'RL_Long_Old_Spearman', 'LR_Short_Old_Spearman', 'RL_Short_Old_Spearman', 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Short_ActuallyIncludedAclus', 'LR_Short_rel_num_cells', 'RL_Short_ActuallyIncludedAclus', 'RL_Short_rel_num_cells', 'LR_Long_Z', 'RL_Long_Z', 'LR_Short_Z', 'RL_Short_Z']
# active_epochs_df.columns
# accumulated_evidence_df = pd.DataFrame({'LR_evidence': accumulated_evidence['Normed_LR_rate'], 'RL_evidence': accumulated_evidence['Normed_LR_rate']}) epoch_accumulated_evidence.items()
epoch_accumulated_evidence_L, epoch_rate_dfs_L, epochs_df_L = epoch_directionality_active_set_evidence(decoders_dict, rank_order_results.LR_ripple.epochs_df)
epochs_df_L

,start,stop,label,duration,LR_Long_ActuallyIncludedAclus,LR_Long_rel_num_cells,RL_Long_ActuallyIncludedAclus,RL_Long_rel_num_cells,LR_Short_ActuallyIncludedAclus,LR_Short_rel_num_cells,RL_Short_ActuallyIncludedAclus,RL_Short_rel_num_cells,LR_evidence,RL_evidence,LR_product_evidence,RL_product_evidence,normed_LR_evidence,normed_RL_evidence,normed_product_LR_evidence,normed_product_RL_evidence
0,40.187177,40.487557,0,0.300380,"[32, 36, 41, 45, 48, 53, 59, 62, 63, 78, 102, ...",12,"[32, 36, 41, 45, 48, 53, 59, 62, 63, 78, 102, ...",12,"[32, 36, 41, 45, 48, 53, 59, 62, 63, 78, 102, ...",12,"[32, 36, 41, 45, 48, 53, 59, 62, 63, 78, 102, ...",12,6.967361,5.032639,0.000000,0.000000,0.580613,0.419387,NaN,NaN
1,41.011916,41.357670,1,0.345753,"[7, 9, 32, 36, 41, 45, 48, 51, 53, 57, 62, 64,...",20,"[7, 9, 32, 36, 41, 45, 48, 51, 53, 57, 62, 64,...",20,"[7, 9, 32, 36, 41, 45, 48, 51, 53, 57, 62, 64,...",20,"[7, 9, 32, 36, 41, 45, 48, 51, 53, 57, 62, 64,...",20,10.309980,8.690020,0.000000,0.000000,0.542631,0.457369,NaN,NaN
2,43.429702,43.482632,2,0.052931,"[48, 51, 53, 55, 59, 62, 78, 81, 88, 108]",10,"[48, 51, 53, 55, 59, 62, 78, 81, 88, 108]",10,"[48, 51, 53, 55, 59, 62, 78, 81, 88, 108]",10,"[48, 51, 53, 55, 59, 62, 78, 81, 88, 108]",10,6.271663,2.728337,0.016934,0.000000,0.696851,0.303149,1.000000,0.000000
4,44.588736,44.829950,4,0.241213,"[9, 31, 32, 36, 48, 51, 57, 59, 62, 63, 88, 98...",13,"[9, 31, 32, 36, 48, 51, 57, 59, 62, 63, 88, 98...",13,"[9, 31, 32, 36, 48, 51, 57, 59, 62, 63, 88, 98...",13,"[9, 31, 32, 36, 48, 51, 57, 59, 62, 63, 88, 98...",13,6.929572,6.070428,0.000000,0.000000,0.533044,0.466956,NaN,NaN
5,45.137119,45.232842,5,0.095723,"[41, 48, 57, 59, 62, 78, 86, 88, 93, 108]",10,"[41, 48, 57, 59, 62, 78, 86, 88, 93, 108]",10,"[41, 48, 57, 59, 62, 78, 86, 88, 93, 108]",10,"[41, 48, 57, 59, 62, 78, 86, 88, 93, 108]",10,5.422436,4.577564,0.000000,0.000085,0.542244,0.457756,0.000000,1.000000
6,45.379714,45.477250,6,0.097536,"[31, 32, 45, 50, 75, 86, 90, 92, 93]",9,"[31, 32, 45, 50, 75, 86, 90, 92, 93]",9,"[31, 32, 45, 50, 75, 86, 90, 92, 93]",9,"[31, 32, 45, 50, 75, 86, 90, 92, 93]",9,3.268585,5.731415,0.000000,0.006670,0.363176,0.636824,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,2033.590214,2033.962356,620,0.372142,"[5, 9, 50, 57, 61, 62, 75, 81, 84, 86, 108]",11,"[5, 9, 50, 57, 61, 62, 75, 81, 84, 86, 108]",11,"[5, 9, 50, 57, 61, 62, 75, 81, 84, 86, 108]",11,"[5, 9, 50, 57, 61, 62, 75, 81, 84, 86, 108]",11,4.770001,5.229999,0.000071,0.000484,0.477000,0.523000,0.127713,0.872287
621,2034.656228,2034.750753,621,0.094525,"[41, 48, 57, 59, 61, 62, 69, 86, 88, 100, 108]",11,"[41, 48, 57, 59, 61, 62, 69, 86, 88, 100, 108]",11,"[41, 48, 57, 59, 61, 62, 69, 86, 88, 100, 108]",11,"[41, 48, 57, 59, 61, 62, 69, 86, 88, 100, 108]",11,7.295859,3.704141,0.004659,0.000000,0.663260,0.336740,1.000000,0.000000
622,2043.892191,2044.072118,622,0.179927,"[7, 45, 48, 61, 63, 84, 86, 88, 90, 102]",10,"[7, 45, 48, 61, 63, 84, 86, 88, 90, 102]",10,"[7, 45, 48, 61, 63, 84, 86, 88, 90, 102]",10,"[7, 45, 48, 61, 63, 84, 86, 88, 90, 102]",10,6.030680,3.969320,0.003810,0.000000,0.603068,0.396932,1.000000,0.000000


In [19]:
_bak_laps = deepcopy(rank_order_results.laps_most_likely_result_tuple)

LR_laps_epoch_accumulated_evidence, LR_laps_epoch_rate_dfs, LR_laps_epochs_df = epoch_directionality_active_set_evidence(decoders_dict, rank_order_results.LR_laps.epochs_df)
RL_laps_epoch_accumulated_evidence, RL_laps_epoch_rate_dfs, RL_laps_epochs_df = epoch_directionality_active_set_evidence(decoders_dict, rank_order_results.RL_laps.epochs_df)

rank_order_results.laps_most_likely_result_tuple.directional_likelihoods_tuple = DirectionalRankOrderLikelihoods(long_relative_direction_likelihoods=LR_laps_epochs_df['normed_LR_evidence'], short_relative_direction_likelihoods=LR_laps_epochs_df['normed_RL_evidence'],
                                 long_best_direction_indices=(LR_laps_epochs_df['normed_LR_evidence']>=LR_laps_epochs_df['normed_RL_evidence']), 
                                 short_best_direction_indices=(LR_laps_epochs_df['normed_LR_evidence']>=LR_laps_epochs_df['normed_RL_evidence']))


rank_order_results.laps_most_likely_result_tuple

,start,stop,label,duration,LR_Long_ActuallyIncludedAclus,LR_Long_rel_num_cells,RL_Long_ActuallyIncludedAclus,RL_Long_rel_num_cells,LR_Short_ActuallyIncludedAclus,LR_Short_rel_num_cells,RL_Short_ActuallyIncludedAclus,RL_Short_rel_num_cells,LR_evidence,RL_evidence,LR_product_evidence,RL_product_evidence,normed_LR_evidence,normed_RL_evidence,normed_product_LR_evidence,normed_product_RL_evidence
0,5.664919,17.359802,0,11.694883,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,17.909030,14.090970,0.000000e+00,0.0,0.559657,0.440343,NaN,NaN
1,31.961164,39.764808,1,7.803644,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,18.909030,14.090970,0.000000e+00,0.0,0.573001,0.426999,NaN,NaN
2,135.834493,144.148150,2,8.313657,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,19.764238,15.235762,0.000000e+00,0.0,0.564693,0.435307,NaN,NaN
3,161.553817,167.318145,3,5.764329,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,19.423985,14.576015,0.000000e+00,0.0,0.571294,0.428706,NaN,NaN
4,234.470126,239.718943,4,5.248817,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,16.354374,11.645626,0.000000e+00,0.0,0.584085,0.415915,NaN,NaN
5,255.250813,262.688459,5,7.437646,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,18.423985,14.576015,0.000000e+00,0.0,0.558303,0.441697,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1999.045622,2002.816193,74,3.770571,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,13.106307,11.893693,0.000000e+00,0.0,0.524252,0.475748,NaN,NaN
75,2003.032708,2008.278298,75,5.245591,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,19.302430,13.697570,0.000000e+00,0.0,0.584922,0.415078,NaN,NaN
76,2019.176367,2024.898701,76,5.722334,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,16.308090,13.691910,0.000000e+00,0.0,0.543603,0.456397,NaN,NaN


In [20]:
RL_laps_epochs_df

,start,stop,label,duration,LR_Long_ActuallyIncludedAclus,LR_Long_rel_num_cells,RL_Long_ActuallyIncludedAclus,RL_Long_rel_num_cells,LR_Short_ActuallyIncludedAclus,LR_Short_rel_num_cells,RL_Short_ActuallyIncludedAclus,RL_Short_rel_num_cells,LR_evidence,RL_evidence,LR_product_evidence,RL_product_evidence,normed_LR_evidence,normed_RL_evidence,normed_product_LR_evidence,normed_product_RL_evidence
0,5.664919,17.359802,0,11.694883,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,"[5, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57, 59...",33,17.909030,14.090970,0.000000e+00,0.0,0.559657,0.440343,NaN,NaN
1,31.961164,39.764808,1,7.803644,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,"[5, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55, 57...",34,18.909030,14.090970,0.000000e+00,0.0,0.573001,0.426999,NaN,NaN
2,135.834493,144.148150,2,8.313657,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,"[5, 7, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, ...",36,19.764238,15.235762,0.000000e+00,0.0,0.564693,0.435307,NaN,NaN
3,161.553817,167.318145,3,5.764329,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 51, 53, 55,...",35,19.423985,14.576015,0.000000e+00,0.0,0.571294,0.428706,NaN,NaN
4,234.470126,239.718943,4,5.248817,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,"[5, 31, 32, 41, 45, 48, 50, 53, 55, 61, 63, 64...",29,16.354374,11.645626,0.000000e+00,0.0,0.584085,0.415915,NaN,NaN
5,255.250813,262.688459,5,7.437646,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,"[5, 9, 31, 32, 36, 41, 45, 48, 50, 53, 55, 57,...",34,18.423985,14.576015,0.000000e+00,0.0,0.558303,0.441697,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1999.045622,2002.816193,74,3.770571,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,"[7, 31, 32, 41, 45, 50, 55, 61, 62, 63, 64, 69...",26,13.106307,11.893693,0.000000e+00,0.0,0.524252,0.475748,NaN,NaN
75,2003.032708,2008.278298,75,5.245591,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,"[5, 7, 9, 31, 32, 41, 45, 48, 50, 51, 53, 55, ...",34,19.302430,13.697570,0.000000e+00,0.0,0.584922,0.415078,NaN,NaN
76,2019.176367,2024.898701,76,5.722334,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,"[5, 7, 31, 32, 41, 45, 48, 50, 53, 55, 57, 61,...",31,16.308090,13.691910,0.000000e+00,0.0,0.543603,0.456397,NaN,NaN


In [ ]:
rank_order_results.LR_ripple.epochs_df

In [ ]:
epoch_accumulated_evidence_R, epoch_rate_dfs_R, epochs_df_R = epoch_directionality_active_set_evidence(decoders_dict, rank_order_results.RL_ripple.epochs_df)
epochs_df_R

In [ ]:
epochs_df_L.plot.scatter(x='start', y='normed_LR_evidence') # 'normed_product_LR_evidence'
epochs_df_L.plot.scatter(x='start', y='normed_product_LR_evidence') # 'normed_product_LR_evidence'

In [ ]:
rank_order_results.LR_laps.epochs_df

In [ ]:
rank_order_results.RL_laps.epochs_df

In [ ]:
ActuallyIncludedAclusColumnNames = ['LR_Long_ActuallyIncludedAclus', 'RL_Long_ActuallyIncludedAclus', 'LR_Short_ActuallyIncludedAclus', 'RL_Short_ActuallyIncludedAclus']
LR_laps_epochs_df[ActuallyIncludedAclusColumnNames]

## find uniques in each one:

def uniques(*aclus_lists):
    

In [ ]:
an_aclus_lists = LR_laps_epochs_df.loc[0, ActuallyIncludedAclusColumnNames].to_dict()
# {'LR_Long_ActuallyIncludedAclus': array([  5,  17,  25,  31,  32,  34,  36,  41,  45,  48,  50,  53,  54,  55,  57,  58,  59,  61,  62,  63,  64,  66,  68,  69,  74,  75,  76,  78,  81,  82,  83,  84,  86,  87,  88,  89,  90,  92,  93,  96,  98, 100, 102, 108]),
#  'RL_Long_ActuallyIncludedAclus': array([  3,   5,  10,  11,  14,  15,  16,  21,  31,  32,  33,  35,  36,  37,  41,  45,  48,  49,  50,  53,  55,  57,  59,  60,  61,  62,  63,  64,  69,  70,  71,  73,  75,  76,  78,  81,  83,  84,  86,  88,  89,  90,  92,  93,  98, 100, 102, 107, 108]),
#  'LR_Short_ActuallyIncludedAclus': array([  5,  17,  25,  31,  32,  34,  36,  41,  45,  48,  50,  53,  54,  55,  57,  58,  59,  61,  62,  63,  64,  66,  68,  69,  74,  75,  76,  78,  81,  82,  83,  84,  86,  87,  88,  89,  90,  92,  93,  96,  98, 100, 102, 108]),
#  'RL_Short_ActuallyIncludedAclus': array([  3,   5,  10,  11,  14,  15,  16,  21,  31,  32,  33,  35,  36,  37,  41,  45,  48,  49,  50,  53,  55,  57,  59,  60,  61,  62,  63,  64,  69,  70,  71,  73,  75,  76,  78,  81,  83,  84,  86,  88,  89,  90,  92,  93,  98, 100, 102, 107, 108])}
# an_aclus_lists['LR_Long_ActuallyIncludedAclus']




LR_only, RL_only = bidirectional_setdiff1d(an_aclus_lists['LR_Long_ActuallyIncludedAclus'], an_aclus_lists['RL_Long_ActuallyIncludedAclus'])




In [ ]:
[bidirectional_setdiff1d(a_LR_aclus, a_RL_aclus) for a_LR_aclus, a_RL_aclus in zip(LR_laps_epochs_df['LR_Long_ActuallyIncludedAclus'].to_list(), LR_laps_epochs_df['RL_Long_ActuallyIncludedAclus'].to_list())]


In [ ]:
RL_laps_epochs_df

In [ ]:
# epochs_df_L.plot(x='start', y=['LR_evidence', 'LR_product_evidence'])

In [ ]:
epochs_df_L.plot(x='start', y=['normed_LR_evidence', 'normed_product_LR_evidence'])

In [ ]:
rank_order_results.LR_laps.epochs_df

In [ ]:
epoch_accumulated_evidence, epoch_rate_dfs, epochs_df_L = epoch_directionality_active_set_evidence(decoders_dict, active_epochs_df)
epochs_df_L


In [ ]:
# 'LR_Long_ActuallyIncludedAclus', 'LR_Long_rel_num_cells', 'RL_Long_ActuallyIncludedAclus', 'RL_Long_rel_num_cells', 'LR_Short_ActuallyIncludedAclus', 'LR_Short_rel_num_cells', 'RL_Short_ActuallyIncludedAclus', 'RL_Short_rel_num_cells'
# 
# epoch_rate_dfs

['normed_LR_evidence','normed_RL_evidence','normed_product_LR_evidence','normed_product_RL_evidence']

In [ ]:
long_laps.to_dataframe()

In [ ]:
epoch_rate_df = epoch_rate_dfs[410]
epoch_rate_df

In [ ]:
epoch_accumulated_evidence

In [ ]:
epochs_df_L['normed_LR_evidence'].hist()

# epochs_df_L[['LR_evidence', 'RL_evidence']]/epochs_df_L[['LR_evidence', 'RL_evidence']].sum(axis=1)


In [ ]:
epochs_df_L['normed_product_LR_evidence'].hist()
epochs_df_L['normed_product_RL_evidence'].hist()


In [ ]:
epoch_rate_dfs[188]

In [ ]:
epochs_df_L.plot(x='start', y='normed_LR_evidence')

In [ ]:
# Determine which cells are active in the given epoch:

# Get these cells' firing rates, using 0.0 if it isn't included in the map for one of the directions




In [ ]:
ripple_combined_epoch_stats_df

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df

In [ ]:
np.isnan(ripple_combined_epoch_stats_df.index).any()

In [ ]:
np.isnan(rank_order_results.ripple_combined_epoch_stats_df.index)

In [ ]:
np.isnan(rank_order_results.ripple_combined_epoch_stats_df.label.to_numpy()).all()

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df

# 2023-12-18 - Simpily detect bimodal cells:

In [ ]:
long_pf1D.peak_indicies

In [ ]:

# active_ratemap.n_neurons


In [ ]:

_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
curr_active_pipeline.display('_display_1d_placefields', 'maze1_any')


In [ ]:
from scipy.signal import find_peaks

active_ratemap = deepcopy(long_pf1D.ratemap)
active_tuning_curves = deepcopy(active_ratemap.tuning_curves)
active_ratemap.tuning_curves.shape # (73, 56) - (n_neurons, n_pos_bins)
peaks_list = [find_peaks(active_tuning_curves[i,:])[0] for i in np.arange(active_ratemap.n_neurons)] # [0] outside the find_peaks function gets the location of the peak
peaks_dict = dict(zip(active_ratemap.neuron_ids, peaks_list))
aclu_n_peaks_dict = {k:len(v) for k,v in peaks_dict.items()} # number of peaks ("models" for each aclu)
unimodal_peaks_dict = {k:v for k,v in peaks_dict.items() if len(v) < 2}
unimodal_peaks_dict

In [ ]:
aclu_n_peaks_dict

In [ ]:
unimodal_only_aclus = np.array(list(unimodal_peaks_dict.keys()))
unimodal_only_aclus

In [ ]:
curr_active_pipeline.display('_display_1d_placefields', 'maze1_any', included_unit_neuron_IDs=unimodal_only_aclus, sortby=[0, 1])

## 2023-12-19

5 epochs where we have a higher Z for short than we do for long

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df.label == 148]

In [ ]:
rank_order_results.LR_ripple.epochs_df[rank_order_results.LR_ripple.epochs_df.label == 148]
# rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df.label == 148]

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df['LR_Short_spearman'].abs() > rank_order_results.ripple_combined_epoch_stats_df['LR_Long_spearman'].abs()]

In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[(rank_order_results.ripple_combined_epoch_stats_df['LR_Short_spearman_Z'].abs() > rank_order_results.ripple_combined_epoch_stats_df['LR_Long_spearman_Z'].abs()) & (rank_order_results.ripple_combined_epoch_stats_df['LR_Short_spearman_Z'].abs() > 1.98)]

In [ ]:
## Get old LR_ripple/RL_ripple results
LR_active_result: RankOrderResult = rank_order_results.LR_ripple
LR_active_result_epochs = LR_active_result.epochs_df
LR_active_result_epoch_labels = LR_active_result.epochs_df.label.to_numpy()
LR_active_result_n_epochs = np.shape(LR_active_result.epochs_df)[0]
LR_z_variable_shapes = [np.shape(x) for x in (LR_active_result.long_z_score, LR_active_result.short_z_score, LR_active_result.long_short_z_score_diff)]
assert np.all([LR_active_result_n_epochs == s[0] for s in LR_z_variable_shapes]), f"z-variable shapes should match the number of epochs"


RL_active_result: RankOrderResult = rank_order_results.RL_ripple
RL_active_result_epochs = RL_active_result.epochs_df
RL_active_result_epoch_labels = RL_active_result.epochs_df.label.to_numpy()
print(f'active_result_epoch_labels: {RL_active_result_epoch_labels}')
RL_active_result_n_epochs = np.shape(RL_active_result.epochs_df)[0]
RL_z_variable_shapes = [np.shape(x) for x in (RL_active_result.long_z_score, RL_active_result.short_z_score, RL_active_result.long_short_z_score_diff)]
print(f'RL_z_variable_shapes: {RL_z_variable_shapes}')
assert np.all([RL_active_result_n_epochs == s[0] for s in RL_z_variable_shapes]), f"z-variable shapes should match the number of epochs"

## LR and RL epoch labels must be equal to each other!
assert (np.array_equal(rank_order_results.LR_ripple.epochs_df.label.to_numpy(), rank_order_results.RL_ripple.epochs_df.label.to_numpy())), f"Epoch labels must be equal"
new_result_epochs_df = deepcopy(rank_order_results.LR_ripple.epochs_df)
new_result_epochs_df.reset_index(drop=True)
new_result_df = pd.DataFrame({
'label': deepcopy(RL_active_result.epochs_df.label.to_numpy()),
'LR_Long_Z': rank_order_results.LR_ripple.long_z_score, 
'LR_Short_Z': rank_order_results.LR_ripple.short_z_score,
'LR_LongShort_Zdiff': rank_order_results.LR_ripple.long_short_z_score_diff,
'RL_Long_Z': rank_order_results.RL_ripple.long_z_score, 
'RL_Short_Z': rank_order_results.RL_ripple.short_z_score,
'RL_LongShort_Zdiff': rank_order_results.RL_ripple.long_short_z_score_diff,
 }, index=new_result_epochs_df.index).reset_index(drop=True)

# new_result_epochs_df.update(new_result_df, errors='raise')
# assert np.shape(new_result_epochs_df)[0] == np.shape(rank_order_results.LR_ripple.epochs_df)[0], f"number of rows must not have changed"
# new_result_epochs_df
new_result_df

In [ ]:
interesting_labels = [4, 148]

In [ ]:
new_result_df[new_result_df.label == 4]

In [ ]:
new_result_df[new_result_df['LR_Long_Z'].abs() > 1.97]


In [ ]:
rank_order_results.ripple_combined_epoch_stats_df[rank_order_results.ripple_combined_epoch_stats_df.label == 4]

# 2023-12-19 - Transmition Matricies Version

In [ ]:
from PendingNotebookCode import TransitionMatrixComputations

In [ ]:
# pf1D = deepcopy(global_pf1D)
# pf1D = deepcopy(short_pf1D)
# pf1D = deepcopy(long_pf1D)


decoders_dict = track_templates.get_decoders_dict()
binned_x_transition_matrix_higher_order_list_dict = {}

for a_decoder_name, a_decoder in decoders_dict.items():
	a_pf1D = deepcopy(a_decoder.pf)
	binned_x_transition_matrix_higher_order_list_dict[a_decoder_name] = TransitionMatrixComputations._compute_position_transition_matrix(a_pf1D.xbin_labels, a_pf1D.filtered_pos_df['binned_x'].to_numpy())

binned_x_transition_matrix_higher_order_list_dict

In [ ]:
# a_decoder.dec

long_LR_pf1D_Decoder.active_time_window_centers

In [ ]:
# Visualization ______________________________________________________________________________________________________ #
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability
out = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[2], a_pf1D.xbin_labels, a_pf1D.xbin_labels, name='binned_x_transition_matrix', title="Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)



In [ ]:
from scipy.sparse import csr_matrix

rank_order_results.LR_laps.spikes_df

In [ ]:
def _time_transition_matrix(df, bin_size=0.25): # "0.25S"
	# df["bin"] = pd.cut(df["t_rel_seconds"], bins=pd.timedelta_range(df["t_rel_seconds"].min(), df["t_rel_seconds"].max(), freq=bin_size))
	# df["bin"] = pd.cut(
	# 	pd.to_timedelta(df["t_rel_seconds"], unit="s"),
	# 	bins=pd.timedelta_range(df["t_rel_seconds"].min(), df["t_rel_seconds"].max(), freq=bin_size),
	# )
	def get_bin(seconds):
		bin_number = int(seconds // bin_size)
		return bin_number

	df["bin"] = df["t_rel_seconds"].apply(get_bin)

	transition_matrix = csr_matrix((
		counts, (i, j)
	) for i, (bin, group) in df.groupby(["bin", "aclu"]).agg(counts=("aclu", "count")).iterrows()
		for j, counts in group.iteritems()
	)
	transition_matrix /= transition_matrix.sum(axis=1, keepdims=True)

	return transition_matrix


test_spikes_df = deepcopy(rank_order_results.LR_laps.spikes_df)

test_spikes_df = test_spikes_df.spikes.add_binned_time_column()

# [['t_rel_seconds', 'aclu', 'flat_spike_idx', 'Probe_Epoch_id']]]


test_spikes_df


In [ ]:
transition_matrix = _time_transition_matrix(df=test_spikes_df) #  bin_size="0.25S"
transition_matrix

In [ ]:
long_LR_pf1D_Decoder.time_window_centers
# long_LR_pf1D_Decoder.decode_specific_epochs
long_LR_pf1D_Decoder.p_x_given_n

# time_bin_size = 0.25
time_bin_size = 0.1
manual_time_bin_start_t = curr_active_pipeline.sess.t_start
manual_time_bin_end_t = curr_active_pipeline.sess.t_stop

print(f'time_bin_size: {time_bin_size}, manual_time_bins: (start_t: {manual_time_bin_start_t}, end_t: {manual_time_bin_end_t})')
manual_time_window_edges, manual_time_window_edges_binning_info = compute_spanning_bins(None, bin_size=time_bin_size, variable_start_value=manual_time_bin_start_t, variable_end_value=manual_time_bin_end_t) # np.shape(out_digitized_variable_bins)[0] == np.shape(spikes_df)[0]
# debug_print_1D_bin_infos(manual_time_window_edges, 'manual_time_window_edges')

## Build the new decoder with custom params:
new_decoder_pf_params = deepcopy(active_computation_config.pf_params) # should be a PlacefieldComputationParameters
# override some settings before computation:
new_decoder_pf_params.time_bin_size = time_bin_size

## 1D Decoder
new_decoder_pf1D = active_pf_1D
new_1D_decoder_spikes_df = new_decoder_pf1D.filtered_spikes_df.copy()
new_1D_decoder_spikes_df = new_1D_decoder_spikes_df.spikes.add_binned_time_column(manual_time_window_edges, manual_time_window_edges_binning_info, debug_print=False)
new_1D_decoder = BayesianPlacemapPositionDecoder(new_decoder_pf_params.time_bin_size, new_decoder_pf1D, new_1D_decoder_spikes_df, manual_time_window_edges=manual_time_window_edges, manual_time_window_edges_binning_info=manual_time_window_edges_binning_info, debug_print=False)
new_1D_decoder.compute_all() #  --> n = self.


In [ ]:
long_LR_pf1D_Decoder.unit_specific_time_binned_spike_counts.shape

In [ ]:
df